<a href="https://colab.research.google.com/github/paruliansaragi/Abstractive-Text-Summarization/blob/master/ABS_1004_Fail_and_tfrecord_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!unzip ./bbc-news-summary.zip
!unrar x "./BBC News Summary.rar"
#!pip install pytorch-pretrained-bert

In [0]:
#@title
#!mkdir bert_pretrained_models
#!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip -P bert_pretrained_models/;
#!unzip bert_pretrained_models/uncased_L-12_H-768_A-12.zip -d bert_pretrained_models/

In [0]:
!pip install pyrouge

    100% |████████████████████████████████| 61kB 2.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/75/d3/0c/e5b04e15b6b87c42e980de3931d2686e14d36e045058983599
Successfully built pyrouge


In [0]:
#!mkdir data

In [0]:
#!pip install flair

In [0]:
#@title
vocab_path ="./vocab.txt"
log_root = "./ptr_nw/log"

In [0]:
#@title
import tensorflow as tf
import sys

!test -d texar_repo || git clone https://github.com/asyml/texar.git texar_repo
if not 'texar_repo' in sys.path:
  sys.path += ['texar_repo']
!pip install funcsigs

In [0]:
#@title Imports
import os
import csv
import collections
import sys
#from texar_repo.examples.bert.utils import data_utils, model_utils, tokenization
import importlib
import tensorflow as tf
#import texar as tx 
#from texar_repo.examples.bert import config_classifier as config_downstream
#from texar_repo.texar.utils import transformer_utils
#from texar_repo.examples.transformer.utils import data_utils, utils
#from texar_repo.examples.transformer.bleu_tool import bleu_wrapper
import glob
import random
import struct
import csv
from tensorflow.core.example import example_pb2
import operator
import numpy as np
import pyrouge
np.random.seed(123)

In [0]:
############UTILS#####################

In [0]:
#@title UTILS
def print_results(article, abstract, decoded_output):
  print ("")
  print('ARTICLE:  %s', article)
  print('REFERENCE SUMMARY: %s', abstract)
  print('GENERATED SUMMARY: %s', decoded_output)
  print( "")
      

def make_html_safe(s):
  s.replace("<", "&lt;")
  s.replace(">", "&gt;")
  return s


def rouge_eval(ref_dir, dec_dir):
  r = pyrouge.Rouge155()
  r.model_filename_pattern = '#ID#_reference.txt'
  r.system_filename_pattern = '(\d+)_decoded.txt'
  r.model_dir = ref_dir
  r.system_dir = dec_dir
  logging.getLogger('global').setLevel(logging.WARNING) # silence pyrouge logging
  rouge_results = r.convert_and_evaluate()
  return r.output_to_dict(rouge_results)


def rouge_log(results_dict, dir_to_write):
  log_str = ""
  for x in ["1","2","l"]:
    log_str += "\nROUGE-%s:\n" % x
    for y in ["f_score", "recall", "precision"]:
      key = "rouge_%s_%s" % (x,y)
      key_cb = key + "_cb"
      key_ce = key + "_ce"
      val = results_dict[key]
      val_cb = results_dict[key_cb]
      val_ce = results_dict[key_ce]
      log_str += "%s: %.4f with confidence interval (%.4f, %.4f)\n" % (key, val, val_cb, val_ce)
  print(log_str)
  results_file = os.path.join(dir_to_write, "ROUGE_results.txt")
  print("Writing final ROUGE results to %s..."%(results_file))
  with open(results_file, "w") as f:
    f.write(log_str)


def calc_running_avg_loss(loss, running_avg_loss, summary_writer, step, decay=0.99):
  if running_avg_loss == 0:  # on the first iteration just take the loss
    running_avg_loss = loss
  else:
    running_avg_loss = running_avg_loss * decay + (1 - decay) * loss
  running_avg_loss = min(running_avg_loss, 12)  # clip
  loss_sum = tf.Summary()
  tag_name = 'running_avg_loss/decay=%f' % (decay)
  loss_sum.value.add(tag=tag_name, simple_value=running_avg_loss)
  summary_writer.add_summary(loss_sum, step)
  return running_avg_loss


def write_for_rouge(reference_sents, decoded_words, ex_index,
                    _rouge_ref_dir, _rouge_dec_dir):
  decoded_sents = []
  while len(decoded_words) > 0:
    try:
      fst_period_idx = decoded_words.index(".")
    except ValueError:
      fst_period_idx = len(decoded_words)
    sent = decoded_words[:fst_period_idx + 1]
    decoded_words = decoded_words[fst_period_idx + 1:]
    decoded_sents.append(' '.join(sent))

  # pyrouge calls a perl script that puts the data into HTML files.
  # Therefore we need to make our output HTML safe.
  decoded_sents = [make_html_safe(w) for w in decoded_sents]
  reference_sents = [make_html_safe(w) for w in reference_sents]

  ref_file = os.path.join(_rouge_ref_dir, "%06d_reference.txt" % ex_index)
  decoded_file = os.path.join(_rouge_dec_dir, "%06d_decoded.txt" % ex_index)

  with open(ref_file, "w") as f:
    for idx, sent in enumerate(reference_sents):
      f.write(sent) if idx == len(reference_sents) - 1 else f.write(sent + "\n")
  with open(decoded_file, "w") as f:
    for idx, sent in enumerate(decoded_sents):
      f.write(sent) if idx == len(decoded_sents) - 1 else f.write(sent + "\n")

  #print("Wrote example %i to file" % ex_index)

In [0]:
############UTILS~~END#####################

In [0]:
##########HPS############

In [0]:
#@title Hyperparameters
# Hyperparameters
hidden_dim= 256
emb_dim= 128
batch_size= 8
max_enc_steps=400
max_dec_steps=100
beam_size=4
min_dec_steps=35
vocab_size=50000

lr=0.15
adagrad_init_acc=0.1
rand_unif_init_mag=0.02
trunc_norm_init_std=1e-4
max_grad_norm=2.0

pointer_gen = True
is_coverage = False
cov_loss_wt = 1.0

eps = 1e-12
max_iterations = 500000

use_gpu=True

lr_coverage=0.15

In [0]:
##########HPS~~~~END############

In [0]:
#############DATA PROCESSING################

In [0]:
#@title
#Tokenize and vocabularize
# Use Bert / Texar to tokenize and get a vocab since the WordPiece tokenization
# will be effective for OOV words

#Create Examples (src and tgt)

#Batchify

#Pretrained model for sequence generation ~ init weights and config

#Train and Save

#Rougify and evaluate


In [0]:
#@title
#bert_pretrain_dir = 'bert_pretrained_models/uncased_L-12_H-768_A-12'

#tokenizer = tokenization.FullTokenizer(
#        vocab_file=os.path.join(bert_pretrain_dir, 'vocab.txt'),
#        do_lower_case=True)

In [0]:
#@title
#vocab_size = len(tokenizer.vocab)

In [0]:
#@title
#TOKENIZE then pass the vocab to the Vocab constructor

In [0]:
#@title DataProcessor
class Dataprocessor(object):
  def __init__(self):
    self.data_dir = './'
  
  def get_art_or_sum(self, file):
    return self._read_file(file)
  
  def get_examples(self, article, summary):
    return self._create_examples(article, summary, Bert_usage=False)
  
  def _create_examples(self, src_lines,tgt_lines,set_type, Bert_usage=False):
        examples = [] 
        for i,data in enumerate(zip(src_lines,tgt_lines)):
            guid = "%s-%s" % (set_type, i)
            if set_type == "test" and i == 0:
                continue
            else:
                #print(data)
                if len(data[0])==0 or len(data[1])==0:
                  continue
                src_lines = tokenization.convert_to_unicode(data[0][0])
                tgt_lines = tokenization.convert_to_unicode(data[1][0])
                if Bert_usage == True:
                  examples.append(InputExample(guid=guid, text_a=src_lines,
                                         text_b=tgt_lines))
                else:
                  examples.append(Example(tgt_lines, tgt_lines, tokenizer.vocab))
        return examples
      
  def _read_file(input_file, quotechar=None):
      """Reads a text file."""
      with tf.gfile.Open(input_file, "r") as f:
          reader = csv.reader(f, delimiter="\n", quotechar=quotechar)
          lines = []
          i = 0
          for line in reader:
              lines.append(line)
      return lines

In [0]:
#@title
#get_example = Dataprocessor
#art, summm = get_example().get_art_or_sum(art_pol_path[1]), get_example().get_art_or_sum(sum_pol_path[1])
#art, summm

In [0]:
########FASTAI##################

In [0]:
#@title Fast.ai

from torchtext import data, vocab
tokenizer = data.get_tokenizer('spacy')
TEXT = data.Field(tokenize=tokenizer, lower=True, eos_token='_eos_')


In [0]:
#@title Turn to csv and train test split { form-width: "5px" }
import pandas as pd
import os
import glob

def read_lines(file):
    """Gets the lines from a file.

    Returns
    -------
    str
        the lines of text of the input file
    """
    with open(file, 'r', encoding="utf-8", errors="replace") as fd:
        first_lines = fd.readlines()
    return first_lines

def merge_art_sum(art_folder_path, sum_folder_path, category):
    """Merges first lines of text files in one folder, and
    writes combined lines into new output file

    Parameters
    ----------
    folder_path : str
        String representation of the folder path containing the text files.
    output_filename : str
        Name of the output file the merged lines will be written to.
    """
    # get all text files
    art_txt_files = glob.glob(art_folder_path + "*.txt")
    sum_txt_files = glob.glob(sum_folder_path + "*.txt")
    # get first lines; map to each text file (sorted)
    art_arr = []
    sum_arr = []
    for fle in art_txt_files:
      # open the file and then call .read() to get the text
      art_arr.append(read_lines(fle))
    for fl in sum_txt_files:
      sum_arr.append(read_lines(fl))
    # create dir
    #if not os.path.exists(output_dir) : os.makedirs(output_dir)
    # return dataframe
    art_df = pd.DataFrame({'article':art_arr})
    sum_df = pd.DataFrame({'summary':sum_arr})
    df = pd.merge(sum_df,art_df,left_index=True,right_index=True)
    return df.to_csv(category+'.csv')
  
categories = ['business', 'entertainment', 'politics', 'sport', 'tech']

def art_or_sum(category):
  artpath = './BBC News Summary/News Articles/' + category +'/'
  sumpath = './BBC News Summary/Summaries/' + category +'/'
  return artpath, sumpath

for i in categories:
  art, summ = art_or_sum(i)
  merge_art_sum(art, summ, i)

In [0]:
#@title
pol = pd.read_csv('politics.csv')
biz = pd.read_csv('business.csv')
ent = pd.read_csv('entertainment.csv')
tech = pd.read_csv('tech.csv')
sport = pd.read_csv('sport.csv')

In [0]:
#@title
def strip_sum_art(df):
  df = df.drop('Unnamed: 0', 1)
  df.summary = df.summary.str.strip('[]')
  df.article = df.article.str.strip('[]')
  df.summary = df.summary.replace('\n',' ', regex=True)
  df.article = df.article.replace(r'\\n',' ', regex=True)
  df.article = df.article.str.strip('"').replace('"', '')
  columns_headers = ["article", "summary"]
  df = df.reindex(columns=columns_headers)
  return df

In [0]:
#@title
pol = strip_sum_art(pol)
biz = strip_sum_art(biz)
ent = strip_sum_art(ent)
tech = strip_sum_art(tech)
sport = strip_sum_art(sport)

In [0]:
#@title
pol.head()

In [0]:
#@title
df = pol.merge(biz, how='outer')
df = df.merge(ent, how='outer')
df = df.merge(tech, how='outer')
df = df.merge(sport, how='outer')

In [0]:
#@title
from sklearn.model_selection import train_test_split

trn, val = train_test_split(df, test_size=0.2)

In [0]:
#@title
trn.to_csv('train.csv', index=False)
val.to_csv('valid.csv', index=False)

In [0]:
#@title
trn = pd.read_csv('train.csv')
val = pd.read_csv('valid.csv')
val.head()

In [0]:
#@title
from sklearn.model_selection import train_test_split

trn_data_fields = [("source", TEXT),
                  ("target", TEXT)]

trn, vld = data.TabularDataset.splits(path='./',
                                     train='train.csv', validation='valid.csv',
                                     format='csv', skip_header=True, fields=trn_data_fields)

In [0]:
#@title
print(trn[0].source, trn[0].target)

In [0]:
#@title
pre_trained_vector_type = 'glove.6B.200d' 
TEXT.build_vocab(trn, vectors=pre_trained_vector_type )

In [0]:
#@title
TEXT.vocab.freqs.most_common(10)

In [0]:
#@title
import torch

USE_GPU = torch.cuda.is_available()
print('USE_GPU={}'.format(USE_GPU))
if USE_GPU:
    print('current_device={}'.format(torch.cuda.current_device()))

batch_size = 4

train_iter, val_iter = data.BucketIterator.splits(
                        (trn, vld), batch_sizes=(batch_size, int(batch_size*1.6)),
                        device=(0 if USE_GPU else -1), 
                        sort_key=lambda x: len(x.source),
                        shuffle=True, sort_within_batch=False, repeat=False)

class BatchTuple():
    def __init__(self, dataset, x_var, y_var):
        self.dataset, self.x_var, self.y_var = dataset, x_var, y_var
        
    def __iter__(self):
        for batch in self.dataset:
            x = getattr(batch, self.x_var) 
            y = getattr(batch, self.y_var)                 
            yield (x, y)
            
    def __len__(self):
        return len(self.dataset)
      
train_iter_tuple = BatchTuple(train_iter, "source", "target")
val_iter_tuple = BatchTuple(val_iter, "source", "target")

In [0]:
#@title
next(iter(train_iter_tuple))

In [0]:
#@title
!pip install fastai -U

In [0]:
#@title
from fastai.text import *

model_data = ModelData('./', trn_dl=train_iter_tuple, val_dl=val_iter_tuple)

len(model_data.trn_dl), len(model_data.val_dl), len(TEXT.vocab)

In [0]:
#@title
t, z = next(model_data.trn_dl.__iter__())
t.size(), z.size()

In [0]:
#@title
sample_source = t.transpose(1,0)[0].data.cpu().numpy()
sample_target = z.transpose(1,0)[0].data.cpu().numpy()

print("source:\n%s \n\ncorresponding tensor:\n%s \n" %(' '.join([TEXT.vocab.itos[o] for o in sample_source]), sample_source))
print("target:\n%s \n\ncorresponding tensor:\n%s \n" %(' '.join([TEXT.vocab.itos[o] for o in sample_target]), sample_target))

##Model

The sequence model, consists of:

- single layer encoder-decoder RNNs (biGRU-to-uniGRU)
- feed-forward attention network (bahdanau)
- option for luong global dot attention
- option for teacher forcing
- option for tied embeddings
- option for multi-layer model
- option for regularization (dropout)

In [0]:
#@title
import torch.nn.functional as F
import torch.nn as nn

In [0]:
#@title
class Seq2SeqRNN(nn.Module):
    
    def __init__(self, rnn_type, input_size, embz_size, hidden_size, batch_size,output_size,max_tgt_len,
                 attention_type, tied_weight_type, pre_trained_vector, pre_trained_vector_type, padding_idx,
                 num_layers=1, encoder_drop=(0.0,0.0), decoder_drop=(0.0,0.0), 
                 bidirectional=True, bias=False, teacher_forcing=True):
        
        super().__init__()
        
        rnn_type, attention_type, tied_weight_type = rnn_type.upper(), attention_type.title(), tied_weight_type.lower()
        
        if rnn_type in ['LSTM', 'GRU']: self.rnn_type = rnn_type
        else: raise ValueError("""An invalid option for '--rnn_type' was supplied,
                                    options are ['LSTM', 'GRU']""")
            
        if attention_type in ['Luong', 'Bahdanau']: self.attention_type = attention_type
        else: raise ValueError("""An invalid option for '--attention_type' was supplied,
                                    options are ['Luong', 'Bahdanau']""")
            
        if tied_weight_type in ['three_way', 'two_way']: self.tied_weight_type = tied_weight_type
        else: raise ValueError("""An invalid option for '--tied_weight_type' was supplied,
                                    options are ['three_way', 'two_way']""")
    
                    
        #initialize model parameters            
        self.output_size, self.embz_size, self.hidden_size = output_size, embz_size, hidden_size//2
        self.num_layers, self.input_size, self.max_tgt_len, self.pre_trained_vector = num_layers, input_size, max_tgt_len, pre_trained_vector
        self.bidirectional,self.teacher_forcing, self.pre_trained_vector_type = bidirectional, teacher_forcing, pre_trained_vector_type
        self.encoder_drop, self.decoder_drop, self.padding_idx = encoder_drop, decoder_drop, padding_idx
        
        
        if self.teacher_forcing: self.force_prob = 1.0
        
        #set bidirectional
        if self.bidirectional: self.num_directions = 2
        else: self.num_directions = 1
            
        
        #encoder
        self.encoder_dropout = nn.Dropout(self.encoder_drop[0])
        self.encoder_embedding_layer = nn.Embedding(self.input_size, self.embz_size, padding_idx=self.padding_idx)
        if self.pre_trained_vector: self.encoder_embedding_layer.weight.data.copy_(self.pre_trained_vector.weight.data)
            
        self.encoder_rnn = getattr(nn, self.rnn_type)(
                           input_size=self.embz_size,
                           hidden_size=self.hidden_size,
                           num_layers=self.num_layers,
                           dropout=self.encoder_drop[1], 
                           bidirectional=self.bidirectional)
        self.encoder_vector_layer = nn.Linear(self.hidden_size*self.num_directions,self.embz_size, bias=bias)
        
       #decoder
        self.decoder_dropout = nn.Dropout(self.decoder_drop[0])
        self.decoder_embedding_layer = nn.Embedding(self.input_size, self.embz_size, padding_idx=self.padding_idx)
        self.decoder_rnn = getattr(nn, self.rnn_type)(
                           input_size=self.embz_size,
                           hidden_size=self.hidden_size*self.num_directions,
                           num_layers=self.num_layers,
                           dropout=self.decoder_drop[1]) 
        self.decoder_output_layer = nn.Linear(self.hidden_size*self.num_directions, self.embz_size, bias=bias)
        self.output_layer = nn.Linear(self.embz_size, self.output_size, bias=bias)
        
        #set tied weights: three way tied weights vs two way tied weights
        if self.tied_weight_type == 'three_way':
            self.decoder_embedding_layer.weight  = self.encoder_embedding_layer.weight
            self.output_layer.weight = self.decoder_embedding_layer.weight  
        else:
            if self.pre_trained_vector: self.decoder_embedding_layer.weight.data.copy_(self.pre_trained_vector.weight.data)
            self.output_layer.weight = self.decoder_embedding_layer.weight  
            
        #set attention
        self.encoder_output_layer = nn.Linear(self.hidden_size*self.num_directions, self.embz_size, bias=bias)
        self.att_vector_layer = nn.Linear(self.embz_size+self.embz_size, self.embz_size,bias=bias)
        if self.attention_type == 'Bahdanau':
            self.decoder_hidden_layer = nn.Linear(self.hidden_size*self.num_directions, self.embz_size, bias=bias)
            self.att_score = nn.Linear(self.embz_size,1,bias=bias)

            
    
    def init_hidden(self, batch_size):
        if self.rnn_type == 'LSTM':
            return (V(torch.zeros(self.num_layers*self.num_directions, batch_size, self.hidden_size)),
                    V(torch.zeros(self.num_layers*self.num_directions, batch_size, self.hidden_size)))
        else:
            return V(torch.zeros(self.num_layers*self.num_directions, batch_size, self.hidden_size))
   

    def _cat_directions(self, hidden):
        def _cat(h):
            return torch.cat([h[0:h.size(0):2], h[1:h.size(0):2]], 2)
            
        if isinstance(hidden, tuple):
            # LSTM hidden contains a tuple (hidden state, cell state)
            hidden = tuple([_cat(h) for h in hidden])
        else:
            # GRU hidden
            hidden = _cat(hidden)
        return hidden    
    
    
    def bahdanau_attention(self, encoder_output, decoder_hidden, decoder_input):
        encoder_output = self.encoder_output_layer(encoder_output) 
        encoder_output = encoder_output.transpose(0,1)
        decoder_hidden = decoder_hidden.transpose(0,1)
        att_score = F.tanh(encoder_output + decoder_hidden)
        att_score = self.att_score(att_score)
        att_weight = F.softmax(att_score, dim=1)
        context_vector = torch.bmm(att_weight.transpose(-1, 1), encoder_output).squeeze(1)
        att_vector = torch.cat((context_vector, decoder_input), dim=1)
        att_vector = self.att_vector_layer(att_vector)
        return att_weight.squeeze(-1), att_vector
    
    
    def luong_attention(self, encoder_output, decoder_output):
        encoder_output = self.encoder_output_layer(encoder_output) 
        encoder_output = encoder_output.transpose(0,1)
        decoder_output = decoder_output.transpose(0,1)
        att_score = torch.bmm(encoder_output, decoder_output.transpose(-1,1))
        att_weight = F.softmax(att_score, dim=1)
        context_vector = torch.bmm(att_weight.transpose(-1, 1), encoder_output).squeeze(1)
        att_vector = torch.cat((context_vector, decoder_output.squeeze(1)), dim=1)
        att_vector = self.att_vector_layer(att_vector)
        att_vector = F.tanh(att_vector)
        return att_weight.squeeze(-1), att_vector
        
    def decoder_forward(self, batch_size, encoder_output, decoder_hidden, y=None):
        decoder_input = V(torch.zeros(batch_size).long())  
        output_seq_stack, att_stack = [], []
        
        for i in range(self.max_tgt_len):
            decoder_input = self.decoder_dropout(self.decoder_embedding_layer(decoder_input))
            if self.attention_type == 'Bahdanau':
                if isinstance(decoder_hidden, tuple):
                    prev_hidden = self.decoder_hidden_layer(decoder_hidden[0][-1]).unsqueeze(0)
                else:
                    prev_hidden = self.decoder_hidden_layer(decoder_hidden[-1]).unsqueeze(0) 
                att, decoder_input = self.bahdanau_attention(encoder_output, prev_hidden, decoder_input)
                decoder_output, decoder_hidden = self.decoder_rnn(decoder_input.unsqueeze(0), decoder_hidden)
                decoder_output = self.decoder_output_layer(decoder_output.squeeze(0)) 
            else:
                decoder_output, decoder_hidden = self.decoder_rnn(decoder_input.unsqueeze(0), decoder_hidden)
                decoder_output = self.decoder_output_layer(decoder_output) 
                att, decoder_output = self.luong_attention(encoder_output, decoder_output)
            att_stack.append(att)
            output = self.output_layer(decoder_output)
            output_seq_stack.append(output)
            decoder_input = V(output.data.max(1)[1])
            if (decoder_input==1).all(): break 
            if self.teacher_forcing:    
                samp_prob = round(random.random(),1)
                if (y is not None) and (samp_prob < self.force_prob):
                    if i >= len(y): break
                    decoder_input = y[i] 
                
        return torch.stack(output_seq_stack), torch.stack(att_stack)
        
                
    def forward(self, seq, y=None):
        batch_size = seq[0].size(0)
        encoder_hidden = self.init_hidden(batch_size)
        encoder_input = self.encoder_dropout(self.encoder_embedding_layer(seq))
        encoder_output, encoder_hidden = self.encoder_rnn(encoder_input, encoder_hidden) 
        if self.bidirectional:
            encoder_hidden = self._cat_directions(encoder_hidden)
        output = self.decoder_forward(batch_size, encoder_output, encoder_hidden, y=y)
        if isinstance(encoder_hidden, tuple):
            encoder_vector = self.encoder_vector_layer(encoder_hidden[0][-1])
        else:
            encoder_vector = self.encoder_vector_layer(encoder_hidden[-1])
        output = output + (encoder_vector,)  
        return output

In [0]:
#@title
its = [next(model_data.trn_dl.__iter__())[1] for i in range(10)]
max_tgt_len = int(np.percentile([its[o].size()[0] for o in range(len(its))], 99))
max_tgt_len

711

In [0]:
#@title
def save_pickle(path, filename, file):
    """Function to save file as pickle"""
    with open(f'{path}/{filename}', 'wb') as f:
        pickle.dump(file, f)

        
def norm_pre_trained_embeddings(vecs, itos, em_sz, padding_idx):
    """Function to load and normalize pretrained vectors"""
    emb = nn.Embedding(len(itos), em_sz, padding_idx=padding_idx)
    wgts = emb.weight.data
    for i,w in enumerate(itos):
        try: 
            wgts[i] = torch.from_numpy(vecs[w]-vec_mean)
            wgts[i] = torch.from_numpy(vecs[w]/vec_std)
        except: pass 
    emb.weight.requires_grad = False    
    return emb


def embedding_param(path, data_field, pre_trained_vector_type, embz_size=128, save_vocab=False, itos='itos', stoi='stoi'):
    """Returns embedding parameters"""
    pre_trained=None
    padding_idx = data_field.vocab.stoi['<pad>']
    index_to_string, string_to_index = data_field.vocab.itos, data_field.vocab.stoi
    if save_vocab:
        vocab_path = os.path.join(path, "vocab")
        os.makedirs(vocab_path, exist_ok=True)
        save_pickle(vocab_path, f'{itos}.pk', index_to_string) 
        save_pickle(vocab_path, f'{stoi}.pk', string_to_index) 
    if pre_trained_vector_type:
        vec_mean, vec_std = data_field.vocab.vectors.numpy().mean(), data_field.vocab.vectors.numpy().std()
        print('pre_trained_vector_mean = %s, pre_trained_vector_std = %s'%(vec_mean, vec_std))
        vector_weight_matrix = data_field.vocab.vectors
        embz_size = vector_weight_matrix.size(1)
        pre_trained = norm_pre_trained_embeddings(vector_weight_matrix, index_to_string, embz_size, padding_idx)
        print('Normalizing.... \npre_trained_vector_mean = %s, pre_trained_vector_std = %s' %(pre_trained.weight.data.numpy().mean(), pre_trained.weight.data.numpy().std()))
    return pre_trained, embz_size, padding_idx

In [0]:
#@title
rev=0
rev += 1
print("rev = %s" %rev)

pre_trained_vector,  embz_size, padding_idx = embedding_param('./', TEXT, pre_trained_vector_type, save_vocab=True)

input_size = len(TEXT.vocab)
hidden_size = 400
output_size =  len(TEXT.vocab)
rnn_type = 'gru'
tied_weight_type ='three_way'
max_tgt_len = max_tgt_len

In [0]:
#@title
!pip install tensorboardX

In [0]:
#@title
from fastai.nlp import *
from fastai.model import Stepper

class Seq2SeqStepper(Stepper):
    def step(self, xs, y, epoch):
        output = self.m(*xs, y)
        xtra = []
        if isinstance(output,tuple): output,*xtra = output
        self.opt.zero_grad()
        loss = raw_loss = self.crit(output, y)
        if self.reg_fn: loss = self.reg_fn(output, xtra, raw_loss)
        loss.backward()
        if self.clip:   # Gradient clipping
            nn.utils.clip_grad_norm(trainable_params_(self.m), self.clip)
        self.opt.step()
        return raw_loss.data
    
def seq2seq_loss(input, target):
    sl,bs = target.size()
    sl_in,bs_in,nc = input.size()
    if sl>sl_in: input = F.pad(input, (0,0,0,0,0,sl-sl_in))
    input = input[:sl]
    return F.cross_entropy(input.view(-1,nc), target.view(-1))#, ignore_index=1)
  
#custom callbacks
from tensorboardX import SummaryWriter
from fastai.sgdr import Callback, DecayScheduler
from fastai.learner import Learner

class TensorboardLogger(Callback):
    def __init__(self, path, log_name, metrics_names=[]):
        super().__init__()
        self.metrics_names = ["validation_loss"]
        self.metrics_names += metrics_names
        log_path = os.path.join(path, "logs")
        self.log_dir = os.path.join(log_path, log_name)
        if os.path.exists(self.log_dir): shutil.rmtree(self.log_dir)
        os.makedirs(self.log_dir)
        
    def on_train_begin(self):
        self.iteration = 0
        self.epoch = 0
        self.writer = SummaryWriter(log_dir=self.log_dir)
    def on_batch_begin(self): pass
    def on_phase_begin(self): pass
    def on_epoch_end(self, metrics):
        self.epoch += 1
        for val, name in zip(metrics, self.metrics_names):
            self.writer.add_scalar(name, val, self.iteration) 
                        
    def on_phase_end(self): pass
    def on_batch_end(self, loss):
        self.iteration += 1
        self.writer.add_scalar("training_loss", loss, self.iteration)
    def on_train_end(self):
        self.writer.close()
        
        
class BestModelCheckPoint(Callback):
    def __init__(self, learner, path, model_name, lr):
        super().__init__()
        self.learner = learner
        self.model_name = model_name
        self.learning_rate = lr
        self.model_log = {}
        self.model_path = self.learner.models_path
        os.makedirs(self.model_path, exist_ok=True)

    def on_train_begin(self): 
        self.first_epoch = True
        self.epoch = 0
        self.best_loss = 0.

    def on_batch_begin(self): pass
    def on_phase_begin(self): pass
    def on_epoch_end(self, metrics): 
        self.epoch += 1
        self.val_loss = metrics[0]
        if self.first_epoch:
            self.best_loss = self.val_loss
            self.first_epoch = False
        elif self.val_loss < self.best_loss:
            self.best_loss = self.val_loss
            self.learner.save(self.model_name)
            self.model_log['training_loss'] = [str(self.train_losses)]
            self.model_log['validation_loss'] = [str(self.val_loss)]
            self.model_log['epoch_num'] = [str(self.epoch)]
            self.model_log['learning_rate'] = [str(self.learning_rate)]
            self.model_log['model_info'] = [w for s in [str(self.learner.model)] for w in s.split('\n')]
            self.model_log['model_info'].append("(attention_type): %s" %self.learner.model.attention_type)
            self.model_log['model_info'].append("(weight_tie): %s" %self.learner.model.tied_weight_type)
            self.model_log['model_info'].append("(pre_trained_vector_type): %s" %self.learner.model.pre_trained_vector_type)
            self.model_log['model_info'].append("(teacher_forcing): %s" %self.learner.model.teacher_forcing)
            if self.learner.model.teacher_forcing: self.model_log['model_info'].append("(teacher_forcing_prob): %s" %self.learner.model.force_prob)
            with open(f'{self.model_path}/{self.model_name}_model_log.json', 'w') as d: json.dump(self.model_log, d)
        else: pass        
    def on_phase_end(self): pass
    def on_batch_end(self, loss):
        self.train_losses = loss
    def on_train_end(self): 
            self.learner.save(f'{self.model_name}_train_end')
            self.model_log['training_loss'] = [str(self.train_losses)]
            self.model_log['validation_loss'] = [str(self.val_loss)]
            self.model_log['epoch_num'] = [str(self.epoch)]
            self.model_log['learning_rate'] = [str(self.learning_rate)]
            self.model_log['model_info'] = [w for s in [str(self.learner.model)] for w in s.split('\n')]
            self.model_log['model_info'].append("(attention_type): %s" %self.learner.model.attention_type)
            self.model_log['model_info'].append("(weight_tie): %s" %self.learner.model.tied_weight_type)
            self.model_log['model_info'].append("(pre_trained_vector_type): %s" %self.learner.model.pre_trained_vector_type)
            self.model_log['model_info'].append("(teacher_forcing): %s" %self.learner.model.teacher_forcing)
            if self.learner.model.teacher_forcing: self.model_log['model_info'].append("(teacher_forcing_prob): %s" %self.learner.model.force_prob)
            with open(f'{self.model_path}/{self.model_name}_train_end_model_log.json', 'w') as d: json.dump(self.model_log, d)

class TeacherForcingSched(Callback):
    def __init__(self, learner, scheduler):
        super().__init__()
        self.learner = learner
        self.scheduler = scheduler
        
    def on_train_begin(self): 
        self.learner.model.force_prob = round(self.scheduler.next_val(),1)
        
    def on_batch_begin(self): pass
    def on_phase_begin(self): pass
    def on_epoch_end(self, metrics): 
        self.learner.model.force_prob = round(self.scheduler.next_val(),1)
        
    def on_phase_end(self): pass
    def on_batch_end(self, loss):pass
    def on_train_end(self): pass

In [0]:
#@title
attention_type='luong'
model_luong = Seq2SeqRNN(rnn_type, input_size, embz_size, hidden_size, 1, output_size, 20,
               attention_type, tied_weight_type, pre_trained_vector, pre_trained_vector_type, padding_idx)

print('='*100)
print('Model log:')
print(model_luong, '\n')
print('- attention_type = {} \n'.format(model_luong.attention_type))
print('- weight_tie = {} \n'.format(model_luong.tied_weight_type))
print('- teacher_forcing = {} \n '.format(model_luong.teacher_forcing)) 
print('- pre_trained_embedding = {} \n'.format(model_luong.pre_trained_vector_type)) 
print('='*100 + '\n')

if USE_GPU:
    model_luong.cuda()
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
learn_luong = RNN_Learner(model_data, SingleModel(model_luong), opt_fn=opt_fn)
learn_luong.crit = seq2seq_loss

In [0]:
#@title
learn_luong.lr_find()
learn_luong.sched.plot()

In [0]:
#@title
# Luong Attention model
lr=1e-3
model_name = f'{model_luong.rnn_type}_{model_luong.attention_type}_rev_{rev}'.lower()
cycle_len=15
best_model = BestModelCheckPoint(learn_luong, model_data.path, model_name, lr)
tb_logger = TensorboardLogger(model_data.path, model_name)
sched = DecayScheduler(DecayType.LINEAR, cycle_len, 0.5, 0.1)
teach_forcer = TeacherForcingSched(learn_luong, sched)
learn_luong.fit(lr, 1, cycle_len=cycle_len, use_clr=(20,10), stepper=Seq2SeqStepper, \
          callbacks=[tb_logger, teach_forcer, best_model])

In [0]:
#@title
import matplotlib.ticker as ticker
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    fontdict = {'fontsize': 14}
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    plt.show()
    
    
def generate(x, y, m):
    probs = m.model(V(x))
    preds, attention, encoder_embedding = to_np(probs[0].max(2)[1]), to_np(probs[1].squeeze(1)), to_np(probs[2])
    sentence = ' '.join([index_to_string[o] for o in x[:,0].data.cpu().numpy() if o != 1])
    result = ' '.join([index_to_string[o] for o in preds[:,0] if o!=1])
    orig = ' '.join([index_to_string[o] for o in y[:,0].data.cpu().numpy() if o != 1])
    print('Input: {}'.format(sentence), '\n')
    print('Original summary: {}'.format(orig), '\n')
    print('Predicted summary: {}'.format(result))
    attention_plot = attention[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))
    return preds, attention, encoder_embedding
  
attention_type='luong'
model_luong = Seq2SeqRNN(rnn_type, input_size, embz_size, hidden_size, batch_size, output_size, max_tgt_len,
               attention_type, tied_weight_type, pre_trained_vector, pre_trained_vector_type, padding_idx)
if USE_GPU:
    model_luong.cuda()
learn_luong = RNN_Learner(model_data, SingleModel(model_luong))
learn_luong.load('gru_luong_rev_1_train_end')

attention_type='bahdanau'
model_bahdanau = Seq2SeqRNN(rnn_type, input_size, embz_size, hidden_size, batch_size, output_size, max_tgt_len,
               attention_type, tied_weight_type, pre_trained_vector, pre_trained_vector_type, padding_idx)
if USE_GPU:
    model_bahdanau.cuda()
learn_bahdanau = RNN_Learner(model_data, SingleModel(model_bahdanau))
learn_bahdanau.load('gru_bahdanau_rev_1_train_end')

# Luong (Global Dot) Attention
x,y = next(iter(model_data.trn_dl))
for i in range(1):
    print(i)
    preds, attention, encoder_embedding = generate(x.transpose(1,0)[i].unsqueeze(1), y.transpose(1,0)[i].unsqueeze(1), learn_luong)

In [0]:
#@title
# Bahdanau Attention
for i in range(1):
    print(i)
    preds, attention, encoder_embedding = generate(x.transpose(1,0)[i].unsqueeze(1), y.transpose(1,0)[i].unsqueeze(1), learn_bahdanau)

In [0]:
#########FASTAI~~END~~~~~~#########

In [0]:
#@title train_val_test_split
def train_val_test_split(processor, summary, article, split_pct):
  max_len = len(summary)
  split_point = int(max_len * split_pct)
  test_split_point = int((max_len - split_point) / 2)
  for (i, path) in enumerate(summary):
    if i < split_point:
      trn_sum_examples, trn_art_examples = read_files(path[i]), read_files(article[i])
    elif i > split_point and i < split_point + test_split_point:
      val_sum_examples, val_art_examples = read_files(path[i]), read_files(article[i])
    else:
      test_sum_examples, test_art_examples = read_files(path[i]), read_files(article[i])
      
  return trn_sum_examples, trn_sum_examples, val_sum_examples, val_art_examples, test_sum_examples, test_art_examples

In [0]:
#@title IO open file
import io
def read_io_file(input_file):
  lines=[]
  with io.open(input_file,'r',encoding='utf-8',errors='ignore') as infile:
      for line in infile:
          lines.append(line.split())
  return lines

In [0]:
#@title Read_files
def read_files(input_file, quotechar=None):
      """Reads a text file."""
      with tf.gfile.Open(input_file, "r") as f:
          reader = csv.reader(f, delimiter="\n", quotechar=quotechar)
          lines = []
          i = 0
          for line in reader:
            if line == "":
              continue
            else:
              lines.append(line)
      return lines

In [0]:
#@title split via loops
trn_arts, val_arts, test_arts = [], [], []
trn_sums, val_sums, test_sums = [], [], []
max_len = len(sum_pol_path)
split_point = int(max_len * 0.8)
test_split_point = int((max_len - split_point) / 2)
for (i, path) in enumerate(sum_pol_path):
  if i < split_point:
    trn_sums.append(read_files(path))
  elif i > split_point and i < split_point + test_split_point:
    val_sums.append(read_files(path))
  else:
    test_sums.append(read_files(path))

In [0]:
#@title split via loops article
for (i, path) in enumerate(art_pol_path):
  if i < split_point:
    trn_arts.append(read_files(path))
  elif i > split_point and i < split_point + test_split_point:
    val_arts.append(read_files(path))
  else:
    test_arts.append(read_files(path))

In [0]:
art_pol_path = glob.glob('./BBC News Summary/News Articles/politics/' +'*.txt')
art_biz_path = glob.glob('./BBC News Summary/News Articles/business/' +'*.txt')
art_tech_path = glob.glob('./BBC News Summary/News Articles/tech/' +'*.txt')
art_sport_path = glob.glob('./BBC News Summary/News Articles/sport/' +'*.txt')
art_ent_path = glob.glob('./BBC News Summary/News Articles/entertainment/' +'*.txt')

sum_pol_path = glob.glob('./BBC News Summary/Summaries/politics/' +'*.txt')
sum_biz_path = glob.glob('./BBC News Summary/Summaries/business/' +'*.txt')
sum_tech_path = glob.glob('./BBC News Summary/Summaries/tech/' +'*.txt')
sum_sport_path = glob.glob('./BBC News Summary/Summaries/sport/' +'*.txt')
sum_ent_path = glob.glob('./BBC News Summary/Summaries/entertainment/' +'*.txt')

In [0]:
sum_pth, art_pth = [], []
for i in categories:
  art_pth.append(os.path.join('./BBC News Summary/News Articles/{}/'.format(i), '*.txt'))
  sum_pth.append(os.path.join('./BBC News Summary/Summaries/{}/'.format(i), '*.txt'))
  #for path in glob.glob():
    

In [0]:
glob_art, glob_sum = [], []
    #for sum_path in glob.glob():
for j in art_pth:
  glob_art.append(glob.glob(j))
for m in sum_pth:
  glob_sum.append(glob.glob(m))

In [0]:
glob_art = flatten_list(glob_art)
glob_sum = flatten_list(glob_sum)

In [0]:
all_arts, all_sums = [], []
for i in glob_sum:
  all_sums.append(read_io_file(i))
for j in glob_art:
  all_arts.append(read_io_file(j))

In [0]:
#@title Combined
combined_trn = list(all_arts) + list(all_sums) 

In [0]:
import keras

from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(cmb)
sequences = tokenizer.texts_to_sequences(cmb)


In [0]:
len(sequences)

1221986

In [0]:
tokenizer.word_index['election']

119

In [0]:
i2w = tokenizer.index_word

In [0]:
for i in cmb[5]:
  print(token2idx(i, vocab))

4995
12


In [0]:
bb = [tokenizer.word_index[sent.lower()] for sent in cmb]

In [0]:
#@title Padding TOkens
# <s> and </s> are used in the data files to segment the abstracts into sentences. They don't receive vocab ids.
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'

PAD_TOKEN = '[PAD]' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNKNOWN_TOKEN = '[UNK]' # This has a vocab id, which is used to represent out-of-vocabulary words
START_DECODING = '[START]' # This has a vocab id, which is used at the start of every decoder input sequence
STOP_DECODING = '[STOP]' # This has a vocab id, which is used at the end of untruncated target sequences

# Note: none of <s>, </s>, [PAD], [UNK], [START], [STOP] should appear in the vocab file.


In [0]:
#@title Flatten List { form-width: "30px" }
def flatten_list(a_list):
  flat_list = [item for sublist in a_list for item in sublist]
  return flat_list

In [0]:
#@title Vocab builder
import pprint as pprint
from collections import Counter
# for tokenizer
import re

def tokenizer(sentences):
    tokens = re.findall(r"[\w]+|[^\s\w]", sentence)
    return tokens

def build_vocab(sentences, is_target=False, max_vocab_size=None):
    word_counter = Counter()
    vocab = dict()
    reverse_vocab = dict()
    
    for sentence in sentences:
        tokens = tokenizer(sentence)
        word_counter.update(tokens)
        
    if max_vocab_size is None:
        max_vocab_size = len(word_counter)
    
    if is_target:
        vocab['START'] = 0
        vocab['PAD'] = 1
        vocab_idx = 2
        for key, value in word_counter.most_common(max_vocab_size):
            vocab[key] = vocab_idx
            vocab_idx += 1
    else:
        vocab['PAD'] = 0
        vocab_idx = 1
        for key, value in word_counter.most_common(max_vocab_size):
            vocab[key] = vocab_idx
            vocab_idx += 1
            
    for key, value in vocab.items():
        reverse_vocab[value] = key
            
    return vocab, reverse_vocab, max_vocab_size

In [0]:
comb = flatten_list(combined_trn)

In [0]:
#comb = [item.decode('utf_8', 'ignore') for item in comb]

In [0]:
#cmb = [x for x in combined_trn]

In [0]:
cmb = flatten_list(comb)

In [0]:
#cmb = cmb.split(' ')
#what = [t.strip() for t in cmb]
#what = [t for t in what if t!=""]

In [0]:
word_counter = Counter()
for sentence in cmb:
  tokens = tokenizer(sentence)
  word_counter.update(tokens)
print(word_counter)

Counter({'the': 65220, '.': 62409, ',': 52761, 'to': 36037, 'of': 28812, '"': 28318, 'and': 25686, 'a': 25034, 'in': 24265, "'": 19219, '-': 18714, 's': 12738, 'for': 12710, 'is': 11921, 'that': 11535, 'said': 11528, 'The': 11494, 'on': 10580, 'was': 9646, 'it': 8246, 'be': 8229, 'has': 7761, 'with': 7436, 'as': 6691, 'have': 6670, 'by': 6419, 'at': 6384, 'he': 6283, 'will': 6260, 'are': 6012, 'I': 5302, 'from': 5060, 'Mr': 5055, 'not': 4973, 'his': 4238, 'would': 3976, 'but': 3855, 'an': 3837, 'had': 3822, 'which': 3748, 'been': 3631, 'they': 3557, ')': 3441, '(': 3436, 'their': 3417, 'year': 3328, 'its': 3247, 'up': 3234, 'more': 3221, 'were': 3150, 'this': 3115, 'who': 3013, 'also': 3009, '%': 2914, 'people': 2891, 'It': 2772, 'about': 2525, 'But': 2517, 'US': 2508, 'we': 2505, ':': 2501, 'than': 2365, 'one': 2361, 'out': 2350, 'after': 2324, 'new': 2292, 'can': 2284, 'or': 2280, 'could': 2203, '£': 2129, 'He': 2110, 'over': 2099, 'first': 1979, 'all': 1972, 'time': 1927, 'last': 18

In [0]:
vocab = dict()
vocab['PAD'] = 0
vocab_idx = 1
for key, value in word_counter.most_common(len(word_counter)):
  vocab[key] = vocab_idx
  vocab_idx += 1

In [0]:
reverse_vocab = dict()
for key, value in vocab.items():
  reverse_vocab[value] = key

In [0]:
#flattened_list = flatten_list(flatten_list(combined_trn))

In [0]:
#inp_voc, inp_rev_voc, inp_max_voc = build_vocab(cmb)

In [0]:
def vocab_to_text_file(vocab):
  voc_txt = []
  for token in vocab:
    voc_txt.append( (token, token2idx(token, vocab)))
  with open('./vocab1.txt', 'w') as f:
    f.write(str(list(voc_txt)))
  return voc_txt

In [0]:
def token2idx(word, vocab):
    return vocab[word]

In [0]:
def idx2token(idx, rev_vocab):
    return rev_vocab[idx]

In [0]:
voc_text = vocab_to_text_file(vocab)

In [0]:
voccy = Vocab('./vocab.txt', 50000, vocab, reverse_vocab)

In [0]:
voccy.word2id('said')

16

In [0]:
voccy.id2word(16)

'said'

In [0]:
token2idx('said', vocab)

16

In [0]:
vocab['said']

16

In [0]:
reverse_vocab[16]

'said'

In [0]:
#@title Split list to words
def split_list_to_words(list_of_list):
  arr = ''
  for i in list_of_list:
    for word in i:
      if word != '':
        arr += word
  return arr

In [0]:
###############ABISEE###########

In [0]:
#@title ABISEE
dm_single_close_quote = u'\u2019' # unicode
dm_double_close_quote = u'\u201d'
END_TOKENS = ['.', '!', '?', '...', "'", "`", '"', dm_single_close_quote, dm_double_close_quote, ")"] # acceptable ways to end a sentence

VOCAB_SIZE = 200000
CHUNK_SIZE = 1000 # num examples per chunk, for the chunked data


def chunk_file(set_name):
  in_file = 'finished_files/%s.bin' % set_name
  reader = open(in_file, "rb")
  chunk = 0
  finished = False
  while not finished:
    chunk_fname = os.path.join(chunks_dir, '%s_%03d.bin' % (set_name, chunk)) # new chunk
    with open(chunk_fname, 'wb') as writer:
      for _ in range(CHUNK_SIZE):
        len_bytes = reader.read(8)
        if not len_bytes:
          finished = True
          break
        str_len = struct.unpack('q', len_bytes)[0]
        example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
        writer.write(struct.pack('q', str_len))
        writer.write(struct.pack('%ds' % str_len, example_str))
      chunk += 1


def chunk_all():
  # Make a dir to hold the chunks
  if not os.path.isdir(chunks_dir):
    os.mkdir(chunks_dir)
  # Chunk the data
  for set_name in ['train', 'val', 'test']:
    print( "Splitting %s data into chunks..." % set_name)
    chunk_file(set_name)
  print( "Saved chunked data in %s" % chunks_dir)



def read_text_file(text_file, if_byte=''):
  lines = []
  with open(text_file, "r") as f:
    for line in f:
      lines.append(line.strip())
    lines=list(filter(None, lines))
  return lines


def hashhex(s):
  """Returns a heximal formated SHA1 hash of the input string."""
  h = hashlib.sha1()
  h.update(s)
  return h.hexdigest()


def get_url_hashes(url_list):
  return [hashhex(url) for url in url_list]


def fix_missing_period(line):
  """Adds a period to a line that is missing a period"""
  if "@highlight" in line: return line
  if line=="": return line
  if line[-1] in END_TOKENS: return line
  # print line[-1]
  return line + " ."


flaggy = False
if flaggy == True:
  if len(sys.argv) != 3:
    print ("USAGE: python make_datafiles.py <cnn_stories_dir> <dailymail_stories_dir>")
    sys.exit()
  cnn_stories_dir = sys.argv[1]
  dm_stories_dir = sys.argv[2]

  # Check the stories directories contain the correct number of .story files
  check_num_stories(cnn_stories_dir, num_expected_cnn_stories)
  check_num_stories(dm_stories_dir, num_expected_dm_stories)

  # Create some new directories
  if not os.path.exists(cnn_tokenized_stories_dir): os.makedirs(cnn_tokenized_stories_dir)
  if not os.path.exists(dm_tokenized_stories_dir): os.makedirs(dm_tokenized_stories_dir)
  if not os.path.exists(finished_files_dir): os.makedirs(finished_files_dir)

  # Run stanford tokenizer on both stories dirs, outputting to tokenized stories directories
  tokenize_stories(cnn_stories_dir, cnn_tokenized_stories_dir)
  tokenize_stories(dm_stories_dir, dm_tokenized_stories_dir)

  # Read the tokenized stories, do a little postprocessing then write to bin files
  write_to_bin(all_test_urls, os.path.join(finished_files_dir, "test.bin"))
  write_to_bin(all_val_urls, os.path.join(finished_files_dir, "val.bin"))
  write_to_bin(all_train_urls, os.path.join(finished_files_dir, "train.bin"), makevocab=True)

  # Chunk the data. This splits each of train.bin, val.bin and test.bin into smaller chunks, each containing e.g. 1000 examples, and saves them in finished_files/chunks
  chunk_all()

##Convert Text to Binary

TFRecord file formats are a TF binary storage format

It stores you data as a sequence of binary strings

You need to specify the structure of your data before writing it to file

TF provides 2 components for this: tf.train.Example and tf.train.SequenceExample. You have to store each sample of your data in one of these structures, then it serializes it and uses a tf.python_io.TFRecordWriter to write it to disk. 

tf.train.example is not a python class but a protocol buffer. It's used to serialize structure data efficiently. 

If you have features where every entry is the same dtype use this. We first create a list that constitute the features. 

**Python strings need to be converted to byes**, text = tf.train.BytesList(value=[b'some text', b'some more text'])

tf.train.Feature wraps a list of data so TF can understand it. text_features = tf.train.Feature(bytes_list=text)

tf.train.Features is a collection of named features. dict = { 'text' : text_features} txt = tf.train.Features(feature=dict)

tf.train.Example is a core component for structuring a TFRecord, it stores features in a single attribute features of type tf.train.Features. exmp = tf.train.Example(features=txt)

tf.python_io.TFRecordWriter accepts a file path and creates a writer object. The write method accepts a string as parameter and writes it to disk, meaning that data must be serialized first. tf.python_io.TFRecordWriter('tffile.tfrecord') as writer: writer.write(examle.SerializeToString())

Reading a TFRecord is easy by using tf.TFRecordReader, define the features you expect in the TFRecord by using tf.FixedLenFeature and tf.VarLenFeature. Parse one tf.train.Example using a tf.parse_single_example (one file). 

https://medium.com/mostly-ai/tensorflow-records-what-they-are-and-how-to-use-them-c46bc4bbb564

In [0]:
def my_get_art_abs(story_file, is_summary=False):
  lines = read_text_file(story_file)

  # Lowercase everything
  lines = [line.lower() for line in lines]

  # Put periods on the ends of lines that are missing them (this is a problem in the dataset because many image captions don't end in periods; consequently they end up in the body of the article as run-on sentences)
  lines = [fix_missing_period(line) for line in lines]

  # Separate out article and abstract sentences
  article_lines = []
  highlights = []
  next_is_highlight = False
  for idx,line in enumerate(lines):
    if line == "":
      continue
    else:
      article_lines.append(line)

  # Make abstract into a single string, putting <s> and </s> tags around the sentences
  if is_summary == True:
    article = ' '.join(["%s %s %s" % (SENTENCE_START, sent, SENTENCE_END) for sent in article_lines])
  else:
    # Make article into a single string
    article = ' '.join(article_lines)

  return article

Given a list of paths, write a file and store tokenized summary or article

Open the file and write it to binary

In [0]:
def unicodify(list_of_strings, is_summary=False):
  tok = []
  lines = [line.lower() for line in list_of_strings]
  lines = [fix_missing_period(line) for line in lines]
  article_lines = []
  
  for idx,line in enumerate(lines):
    if line == "":
      continue
    elif is_summary == True:
      article_lines.append(line)
    else:
      article_lines.append(line)
      
  if is_summary == True:
    #for each sentence join 
    
    article = ' '.join(["%s %s %s" % (SENTENCE_START, sent, SENTENCE_END) for sent in article_lines])
    #Pass in list of lists get a list of strings tokenized back
    #abs_tokens = article.split(' ')
    tok_lines = nltk_tokenizer(article)
    #convert list of strings to bytes
    #strings = list_str_to_byte(tok_lines)
    return article
  else:
    #Pass in list of lists get a list of strings tokenized back
    tok_lines = nltk_tokenizer(article_lines)
    #convert list of strings to bytes
    strings = list_str_to_byte(tok_lines)
    return strings

In [0]:
from keras.preprocessing.text import text_to_word_sequence

def keras_tokenizer(doc):
  string=[]
  for i in doc:
    string.append(text_to_word_sequence(i))
  return string


In [0]:
import nltk
nltk.download('punkt')

def nltk_tokenizer(doc):
  doc = ' '.join([str(elem) for elem in doc ])
  return nltk.word_tokenize(doc)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
#artthing = unicodify( read_text_file('./BBC News Summary/News Articles/politics/068.txt'))
#sumething = unicodify( read_text_file('./BBC News Summary/Summaries/politics/068.txt'))

In [0]:
def list_str_to_byte(list_str):
  strings = []
  for i in list_str:
    strings.append(str.encode(i))
  #strings = [str.encode(item) for item in list_str]
  #strings = ' '.join([str(elem) for elem in doc ])
  return strings

In [0]:
#@title Spacy Tokenizer
def spacy_tokenizer(doc):
  
  doc = ' '.join([str(elem) for elem in doc ])
  words = nlp(doc)
  empt = []
  empty = []
  for word in words:
      empt.append(word)
  for tok in empt:
    empty.append(str.encode(tok))
  return empty

In [0]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

In [0]:
art=read_text_file(art_pol_path[0])
art=unicodify(art)

In [0]:
tok_art, tok_sum = unicodify(read_text_file(art_pol_path[0])), unicodify(read_text_file(sum_pol_path[0]))

In [0]:
tok_sum

In [0]:
totext = tf.train.BytesList(value=art)
totext

In [0]:
tf_example = example_pb2.Example()
tf_example.features.feature['article'].bytes_list.value.extend(art)
tf_example_str = tf_example.SerializeToString()

In [0]:
tf_example.features.feature['article'].bytes_list.value[0]

b'kennedy'

In [0]:
.features.feature['article'].bytes_list.value[0]

In [0]:
tf!rm -r ./tokenized

In [0]:
categories = ['politics', 'business', 'tech', 'sport', 'entertainment']

art_pol_path = glob.glob('./BBC News Summary/News Articles/politics/' +'*.txt')
art_biz_path = glob.glob('./BBC News Summary/News Articles/business/' +'*.txt')
art_tech_path = glob.glob('./BBC News Summary/News Articles/tech/' +'*.txt')
art_sport_path = glob.glob('./BBC News Summary/News Articles/sport/' +'*.txt')
art_ent_path = glob.glob('./BBC News Summary/News Articles/entertainment/' +'*.txt')

sum_pol_path = glob.glob('./BBC News Summary/Summaries/politics/' +'*.txt')
sum_biz_path = glob.glob('./BBC News Summary/Summaries/business/' +'*.txt')
sum_tech_path = glob.glob('./BBC News Summary/Summaries/tech/' +'*.txt')
sum_sport_path = glob.glob('./BBC News Summary/Summaries/sport/' +'*.txt')
sum_ent_path = glob.glob('./BBC News Summary/Summaries/entertainment/' +'*.txt')

In [0]:
pap = os.listdir('./BBC News Summary/News Articles/politics/')
psp = os.listdir('./BBC News Summary/Summaries/politics/')

In [0]:
spacyify(art_pol_path,
         sum_pol_path,
         pap,
         './tokenized',
        makevocab=False)

In [0]:
make_vocab(read_text_file(art_pol_path[1]), read_text_file(sum_pol_path[1]))

Finished writing vocab file


In [0]:
comb = read_text_file(art_pol_path[1]) + read_text_file(sum_pol_path[1])

In [0]:
comb

In [0]:
lines = [line.lower() for line in comb]
lines = [fix_missing_period(line) for line in lines]
article_lines = []

In [0]:
for idx,line in enumerate(lines):
  if line == "":
    continue # empty line
  else:
    article_lines.append(line)
article = ' '.join(article_lines)

In [0]:
arts = article.split(' ')
arts = [t.strip() for t in arts]
arts = [t for t in arts if t!=""]

In [0]:
def make_vocab(arts, sums):
  comb = arts + sums
  lines = [line.lower() for line in comb]
  lines = [fix_missing_period(line) for line in lines]
  article_lines = []
  
  for idx,line in enumerate(lines):
    if line == "":
      continue # empty line
    else:
      article_lines.append(line)
  
  article = ' '.join(article_lines)
    
  vocab_counter = collections.Counter()
  tokens = article.split(' ')
  #art_tokens = arts.split(' ')
  #sum_tokens = sums.split(' ')
  #tokens = [t for t in tokens if t not in [SENTENCE_START, SENTENCE_END]] # remove these tags from vocab
  #tokens = art_tokens + abs_tokens
  tokens = [t.strip() for t in tokens] # strip
  tokens = [t for t in tokens if t!=""] # remove empty
  vocab_counter.update(tokens)
  with open("./vocab.txt", 'w') as writer:
    for word, count in vocab_counter.most_common(VOCAB_SIZE):
      writer.write(word + ' ' + str(count) + '\n')
  print ("Finished writing vocab file")

In [0]:
def spacyify(path_to_art, path_to_sum, a_l_dir, tokenized_stories_dir, makevocab=False):
  #loop over list of files
  #get the list of strings
  #unicodify
  #tokenize
  #write to bin
  max_len = len(path_to_art)
  split_point = int(max_len * 0.8)
  test_split_point = int((max_len - split_point) / 2)
  
  if not os.path.isdir(tokenized_stories_dir):
    os.mkdir(tokenized_stories_dir)
    os.mkdir(tokenized_stories_dir + '/train/')
    os.mkdir(tokenized_stories_dir + '/test/')
    os.mkdir(tokenized_stories_dir + '/val/')
    
  comb_arts = []
  comb_sums = []
  
  for index, (path_art, path_sum) in enumerate(zip(path_to_art, path_to_sum)):
    
    
    
    if index < split_point:
      
      tok_art, tok_sum = unicodify(read_text_file(path_art)), unicodify(read_text_file(path_sum))
      my_write_to_bin(tok_art, tok_sum, os.path.join(tokenized_stories_dir + '/train', a_l_dir[index]))
      comb_arts.append(tok_art)
      comb_sums.append(tok_sum)
    elif index > split_point and index < split_point + test_split_point:
      
      tok_art, tok_sum = unicodify(read_text_file(path_art)), unicodify(read_text_file(path_sum))
      my_write_to_bin(tok_art, tok_sum, os.path.join(tokenized_stories_dir + '/val', a_l_dir[index]))
      comb_arts.append(tok_art)
      comb_sums.append(tok_sum)
    else:
      
      tok_art, tok_sum = unicodify(read_text_file(path_art)), unicodify(read_text_file(path_sum))
      my_write_to_bin(tok_art, tok_sum, os.path.join(tokenized_stories_dir + '/test', a_l_dir[index]))
      comb_arts.append(tok_art)
      comb_sums.append(tok_sum)
    
    if makevocab == True:
        vocab_counter = collections.Counter()
        art_tokens = comb_arts.split(' ')
        sum_tokens = comb_sums.split(' ')
        #tokens = [t for t in tokens if t not in [SENTENCE_START, SENTENCE_END]] # remove these tags from vocab
        tokens = art_tokens + abs_tokens
        tokens = [t.strip() for t in tokens] # strip
        tokens = [t for t in tokens if t!=""] # remove empty
        vocab_counter.update(tokens)
        with open("./vocab", 'w') as writer:
          for word, count in vocab_counter.most_common(VOCAB_SIZE):
            writer.write(word + ' ' + str(count) + '\n')
        print ("Finished writing vocab file")
      

In [0]:
tok_art[41]

b'of'

In [0]:
tok_art.index(b'of')

41

In [0]:
tf_example = example_pb2.Example()
tf_example.features.feature['article'].bytes_list.value.extend(tok_art)
tf_example_str = tf_example.SerializeToString()

In [0]:
g_abstract_sentences = [sent.strip() for sent in abstract2sents(tok_art)]
g_abstract_sentences

[]

In [0]:
#@title Write to bin
def my_write_to_bin(art, summ, out_file, makevocab=False):
  """Reads the tokenized .story files corresponding to the urls listed in the url_file and writes them to a out_file."""

  if makevocab:
    vocab_counter = collections.Counter()

  with open(out_file, 'wb') as writer:
      # get the tokenized articles and convert to binary
      article, abstract = art, summ

      # Write to tf.Example
      tf_example = example_pb2.Example()
      tf_example.features.feature['article'].bytes_list.value.extend(article)
      tf_example.features.feature['abstract'].bytes_list.value.extend(abstract)
      tf_example_str = tf_example.SerializeToString()
      str_len = len(tf_example_str)
      writer.write(struct.pack('q', str_len))
      writer.write(struct.pack('%ds' % str_len, tf_example_str))

      # Write the vocab to file, if applicable
      if makevocab:
        art_tokens = article.split(' ')
        abs_tokens = abstract.split(' ')
        abs_tokens = [t for t in abs_tokens if t not in [SENTENCE_START, SENTENCE_END]] # remove these tags from vocab
        tokens = art_tokens + abs_tokens
        tokens = [t.strip() for t in tokens] # strip
        tokens = [t for t in tokens if t!=""] # remove empty
        vocab_counter.update(tokens)

  print ("Finished writing file %s\n" % out_file)

  # write vocab to file
  if makevocab:
    print( "Writing vocab file...")
    with open(os.path.join(finished_files_dir, "vocab"), 'w') as writer:
      for word, count in vocab_counter.most_common(VOCAB_SIZE):
        writer.write(word + ' ' + str(count) + '\n')
    print ("Finished writing vocab file")

In [0]:
###############ABISEE~~END###########

In [0]:
#############DATA PROCESSING~~END################

In [0]:
def file_based_convert_examples_to_features(
        examples, max_seq_length_src,max_seq_length_tgt,tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.python_io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        #print("ex_index",ex_index)

        if (ex_index+1) %1000 == 0 :
          print("------------processed..{}...examples".format(ex_index))
          
        feature = convert_single_example(ex_index, example,
                                         max_seq_length_src,max_seq_length_tgt,tokenizer)

        def create_int_feature(values):
            return tf.train.Feature(
                int64_list=tf.train.Int64List(value=list(values)))

        features = collections.OrderedDict()
        features["src_input_ids"] = create_int_feature(feature.src_input_ids)
        features["src_input_mask"] = create_int_feature(feature.src_input_mask)
        features["src_segment_ids"] = create_int_feature(feature.src_segment_ids)

        features["tgt_input_ids"] = create_int_feature(feature.tgt_input_ids)
        features["tgt_input_mask"] = create_int_feature(feature.tgt_input_mask)
        features['tgt_labels'] = create_int_feature(feature.tgt_labels)
        
        
        
        #print(feature.tgt_labels)
        

        tf_example = tf.train.Example(
            features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())

Just need to yield example objects??

In [0]:
def my_example_generator(data_path, single_pass):
  while True:
    filelist = glob.glob(data_path) # get the list of datafiles
    assert filelist, ('Error: Empty filelist at %s' % data_path) # check filelist isn't empty
    if single_pass:
      filelist = sorted(filelist)
    else:
      random.shuffle(filelist)
    for f in filelist:
      reader = open(f, 'rb')
      while True:
        len_bytes = reader.read(8)
        if not len_bytes: break # finished reading this file
        str_len = struct.unpack('q', len_bytes)[0]
        example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
        print('Yielding')
        yield example_pb2.Example.FromString(example_str)
    if single_pass:
      print ("example_generator completed reading all datafiles. No more data.")
      break

In [0]:
tf.keras.preprocessing.text.text_to_word_sequence(
    text,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' '
)

In [0]:
genny = my_example_generator(train_data_path, single_pass=False)

In [0]:
next(genny)

In [0]:
 def my_text_generator(example_generator):
    while True:
      e = next(example_generator) # e is a tf.Example
      try:
        article_text = e.features.feature['article'].bytes_list.value[0] # the article text was saved under the key 'article' in the data files
        abstract_text = e.features.feature['abstract'].bytes_list.value[0] # the abstract text was saved under the key 'abstract' in the data files
      except ValueError:
        tf.logging.error('Failed to get article or abstract from example')
        continue
      if len(article_text)==0: # See https://github.com/abisee/pointer-generator/issues/1
        #tf.logging.warning('Found an example with empty article text. Skipping it.')
        continue
      else:
        yield (article_text, abstract_text)

In [0]:
gen2 = my_text_generator(genny)

In [0]:
next(gen2)

Yielding


(b'tories', b'six')

In [0]:
(article, abstract) = next(gen2)

Yielding


In [0]:
abstract

b'and'

In [0]:
abstract.index(b'and')

0

In [0]:
cur = 0
sents = []
i =0
while i<10:
  try:
    start_p = abstract.index(SENTENCE_START, cur)
    end_p = abstract.index(SENTENCE_END, start_p + 1)
    cur = end_p + len(SENTENCE_END)
    sents.append(abstract[start_p+len(SENTENCE_START):end_p])
    i+=1
  except ValueError as e: # no more sentences
    return sents

In [0]:
g_abstract_sentences = [sent for sent in abstract2sents(tok_art)]

In [0]:
g_abstract_sentences

[]

In [0]:
import keras 

keras.preprocessing.text.Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)

Using TensorFlow backend.


In [0]:
def my_abstract2sents():
  cur = 0
  sents = []
  while True:
    try:
      start_p = abstract.index(SENTENCE_START, cur)
      end_p = abstract.index(SENTENCE_END, start_p + 1)
      cur = end_p + len(SENTENCE_END)
      sents.append(abstract[start_p+len(SENTENCE_START):end_p])
    except ValueError as e: # no more sentences
      return sents

In [0]:
abstract_sentences = [sent.strip() for sent in abstract2sents(abstract)]

simply have to create examples and put them in the queue

THis just wants the indexes of the abstract's which are byte encoded.

So can we not just use keras tokenizer and use the word indices from there??

In [0]:
def fill_example_queue(self):
    input_gen = self.text_generator(example_generator(self._data_path, self._single_pass))

    while True:
      try:
        (article, abstract) = next(input_gen) # read the next example from file. article and abstract are both strings.
      except StopIteration: # if there are no more examples:
        tf.logging.info("The example generator for this example queue filling thread has exhausted data.")
        if self._single_pass:
          tf.logging.info("single_pass mode is on, so we've finished reading dataset. This thread is stopping.")
          self._finished_reading = True
          break
        else:
          raise Exception("single_pass mode is off but the example generator is out of data; error.")

      #abstract_sentences = [sent.strip() for sent in data.abstract2sents(abstract)] # Use the <s> and </s> tags in abstract to get a list of sentences.
      abstract_sentences = [sent.strip() for sent in abstract2sents(abstract)]
      example = Example(article, abstract_sentences, self._vocab) # Process into an Example.
      self._example_queue.put(example)

In [0]:
a################VOCAB############

In [0]:
#@title Vocab class
# <s> and </s> are used in the data files to segment the abstracts into sentences. They don't receive vocab ids.
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'

PAD_TOKEN = '[PAD]' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNKNOWN_TOKEN = '[UNK]' # This has a vocab id, which is used to represent out-of-vocabulary words
START_DECODING = '[START]' # This has a vocab id, which is used at the start of every decoder input sequence
STOP_DECODING = '[STOP]' # This has a vocab id, which is used at the end of untruncated target sequences

# Note: none of <s>, </s>, [PAD], [UNK], [START], [STOP] should appear in the vocab file.


class Vocab(object):

  def __init__(self, vocab_file, max_size, inp_vocab=None, rev_voc=None):
    self._word_to_id = {}
    self._id_to_word = {}
    self._count = 0 # keeps track of total number of words in the Vocab

    # [UNK], [PAD], [START] and [STOP] get the ids 0,1,2,3.
    for w in [UNKNOWN_TOKEN, PAD_TOKEN, START_DECODING, STOP_DECODING]:
      self._word_to_id[w] = self._count
      self._id_to_word[self._count] = w
      self._count += 1
      
    # Read the vocab file and add words up to max_size
    for line in inp_vocab:
      w = line
      if w in [SENTENCE_START, SENTENCE_END, UNKNOWN_TOKEN, PAD_TOKEN, START_DECODING, STOP_DECODING]:
        raise Exception('<s>, </s>, [UNK], [PAD], [START] and [STOP] shouldn\'t be in the vocab file, but %s is' % w)
      if w in self._word_to_id:
        raise Exception('Duplicated word in vocabulary file: %s' % w)
      #self._word_to_id[w] = self._count
      self._word_to_id[w] = token2idx(w, inp_vocab)
      #self._id_to_word[self._count] = w
      self._id_to_word[self._word_to_id[w]] = idx2token(self._word_to_id[w], rev_voc)
      self._count += 1
      if max_size != 0 and self._count >= max_size:
        print ("max_size of vocab was specified as %i; we now have %i words. Stopping reading." % (max_size, self._count))
        break

    #print ("Finished constructing vocabulary of %i total words. Last word added: %s" % (self._count, self._id_to_word[self._count-1]))

    # Read the vocab file and add words up to max_size
    #with open(vocab_file, 'r') as vocab_f:
    #  for line in vocab_f:
    #    pieces = line.split()
    #    if len(pieces) != 2:
    #      print ('Warning: incorrectly formatted line in vocabulary file: %s\n' % line)
    #      continue
    #    w = pieces[0]
    #    if w in [SENTENCE_START, SENTENCE_END, UNKNOWN_TOKEN, PAD_TOKEN, START_DECODING, STOP_DECODING]:
    #      raise Exception('<s>, </s>, [UNK], [PAD], [START] and [STOP] shouldn\'t be in the vocab file, but %s is' % w)
    #    if w in self._word_to_id:
    #      raise Exception('Duplicated word in vocabulary file: %s' % w)
    #    self._word_to_id[w] = self._count
    #    self._id_to_word[self._count] = w
    #    self._count += 1
    #    if max_size != 0 and self._count >= max_size:
    #      print ("max_size of vocab was specified as %i; we now have %i words. Stopping reading." % (max_size, self._count))
    #      break

    #print ("Finished constructing vocabulary of %i total words. Last word added: %s" % (self._count, self._id_to_word[self._count-1]))

  def word2id(self, word):
    if word not in self._word_to_id:
      return self._word_to_id[UNKNOWN_TOKEN]
    return self._word_to_id[word]

  def id2word(self, word_id):
    if word_id not in self._id_to_word:
      raise ValueError('Id not found in vocab: %d' % word_id)
    return self._id_to_word[word_id]

  def size(self):
    return self._count

  def write_metadata(self, fpath):
    print ("Writing word embedding metadata file to %s..." % (fpath))
    with open(fpath, "w") as f:
      fieldnames = ['word']
      writer = csv.DictWriter(f, delimiter="\t", fieldnames=fieldnames)
      for i in xrange(self.size()):
        writer.writerow({"word": self._id_to_word[i]})


def example_generator(data_path, single_pass):
  while True:
    filelist = glob.glob(data_path) # get the list of datafiles
    assert filelist, ('Error: Empty filelist at %s' % data_path) # check filelist isn't empty
    if single_pass:
      filelist = sorted(filelist)
    else:
      random.shuffle(filelist)
    for f in filelist:
      reader = open(f, 'rb')
      while True:
        len_bytes = reader.read(8)
        if not len_bytes: break # finished reading this file
        str_len = struct.unpack('q', len_bytes)[0]
        example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
        print('Yielding')
        yield example_pb2.Example.FromString(example_str)
    if single_pass:
      print ("example_generator completed reading all datafiles. No more data.")
      break


def article2ids(article_words, vocab):
  ids = []
  oovs = []
  unk_id = vocab.word2id(UNKNOWN_TOKEN)
  for w in article_words:
    i = vocab.word2id(w)
    if i == unk_id: # If w is OOV
      if w not in oovs: # Add to list of OOVs
        oovs.append(w)
      oov_num = oovs.index(w) # This is 0 for the first article OOV, 1 for the second article OOV...
      ids.append(vocab.size() + oov_num) # This is e.g. 50000 for the first article OOV, 50001 for the second...
    else:
      ids.append(i)
  return ids, oovs


def abstract2ids(abstract_words, vocab, article_oovs):
  ids = []
  unk_id = vocab.word2id(UNKNOWN_TOKEN)
  for w in abstract_words:
    i = vocab.word2id(w)
    if i == unk_id: # If w is an OOV word
      if w in article_oovs: # If w is an in-article OOV
        vocab_idx = vocab.size() + article_oovs.index(w) # Map to its temporary article OOV number
        ids.append(vocab_idx)
      else: # If w is an out-of-article OOV
        ids.append(unk_id) # Map to the UNK token id
    else:
      ids.append(i)
  return ids


def outputids2words(id_list, vocab, article_oovs):
  words = []
  for i in id_list:
    try:
      w = vocab.id2word(i) # might be [UNK]
    except ValueError as e: # w is OOV
      assert article_oovs is not None, "Error: model produced a word ID that isn't in the vocabulary. This should not happen in baseline (no pointer-generator) mode"
      article_oov_idx = i - vocab.size()
      try:
        w = article_oovs[article_oov_idx]
      except ValueError as e: # i doesn't correspond to an article oov
        raise ValueError('Error: model produced word ID %i which corresponds to article OOV %i but this example only has %i article OOVs' % (i, article_oov_idx, len(article_oovs)))
    words.append(w)
  return words


def abstract2sents(abstract):
  cur = 0
  sents = []
  while True:
    try:
      start_p = abstract.index(SENTENCE_START, cur)
      end_p = abstract.index(SENTENCE_END, start_p + 1)
      cur = end_p + len(SENTENCE_END)
      sents.append(abstract[start_p+len(SENTENCE_START):end_p])
    except ValueError as e: # no more sentences
      return sents


def show_art_oovs(article, vocab):
  unk_token = vocab.word2id(UNKNOWN_TOKEN)
  words = article.split(' ')
  words = [("__%s__" % w) if vocab.word2id(w)==unk_token else w for w in words]
  out_str = ' '.join(words)
  return out_str


def show_abs_oovs(abstract, vocab, article_oovs):
  unk_token = vocab.word2id(UNKNOWN_TOKEN)
  words = abstract.split(' ')
  new_words = []
  for w in words:
    if vocab.word2id(w) == unk_token: # w is oov
      if article_oovs is None: # baseline mode
        new_words.append("__%s__" % w)
      else: # pointer-generator mode
        if w in article_oovs:
          new_words.append("__%s__" % w)
        else:
          new_words.append("!!__%s__!!" % w)
    else: # w is in-vocab word
      new_words.append(w)
  out_str = ' '.join(new_words)
  return out_str

In [0]:
################VOCAB~~~EMD############

In [0]:
################BATCHER############

In [0]:
import queue as Queue
import time
from random import shuffle
from threading import Thread

In [0]:
myArt, MySum = read_text_file(art_pol_path[1]), read_text_file(sum_pol_path[1])

In [0]:
myA=split_list_to_words(myArt)
myS=split_list_to_words(MySum)

In [0]:
myA.split()

In [0]:
egg = Example(myA, myS, voccy)

In [0]:
egg.article_oovs

In [0]:
t#@title Example Class
class Example(object):

  def __init__(self, article, abstract_sentences, vocab):
    # Get ids of special tokens
    start_decoding = vocab.word2id(START_DECODING)
    stop_decoding = vocab.word2id(STOP_DECODING)

    # Process the article
    article_words = article.split()
    if len(article_words) > max_enc_steps:
      article_words = article_words[:max_enc_steps]
    self.enc_len = len(article_words) # store the length after truncation but before padding
    self.enc_input = [vocab.word2id(w) for w in article_words] # list of word ids; OOVs are represented by the id for UNK token

    # Process the abstract
    abstract = ' '.join(abstract_sentences) # string
    abstract_words = abstract.split() # list of strings
    abs_ids = [vocab.word2id(w) for w in abstract_words] # list of word ids; OOVs are represented by the id for UNK token

    # Get the decoder input sequence and target sequence
    self.dec_input, self.target = self.get_dec_inp_targ_seqs(abs_ids, max_dec_steps, start_decoding, stop_decoding)
    self.dec_len = len(self.dec_input)

    # If using pointer-generator mode, we need to store some extra info
    if pointer_gen:
      # Store a version of the enc_input where in-article OOVs are represented by their temporary OOV id; also store the in-article OOVs words themselves
      self.enc_input_extend_vocab, self.article_oovs = article2ids(article_words, vocab)

      # Get a verison of the reference summary where in-article OOVs are represented by their temporary article OOV id
      abs_ids_extend_vocab = abstract2ids(abstract_words, vocab, self.article_oovs)

      # Overwrite decoder target sequence so it uses the temp article OOV ids
      _, self.target = self.get_dec_inp_targ_seqs(abs_ids_extend_vocab, max_dec_steps, start_decoding, stop_decoding)

    # Store the original strings
    self.original_article = article
    self.original_abstract = abstract
    self.original_abstract_sents = abstract_sentences


  def get_dec_inp_targ_seqs(self, sequence, max_len, start_id, stop_id):
    inp = [start_id] + sequence[:]
    target = sequence[:]
    if len(inp) > max_len: # truncate
      inp = inp[:max_len]
      target = target[:max_len] # no end_token
    else: # no truncation
      target.append(stop_id) # end token
    assert len(inp) == len(target)
    return inp, target


  def pad_decoder_inp_targ(self, max_len, pad_id):
    while len(self.dec_input) < max_len:
      self.dec_input.append(pad_id)
    while len(self.target) < max_len:
      self.target.append(pad_id)


  def pad_encoder_input(self, max_len, pad_id):
    while len(self.enc_input) < max_len:
      self.enc_input.append(pad_id)
    if pointer_gen:
      while len(self.enc_input_extend_vocab) < max_len:
        self.enc_input_extend_vocab.append(pad_id)

In [0]:
#@title Batch Class
class Batch(object):
  def __init__(self, example_list, vocab, batch_size):
    self.batch_size = batch_size
    self.pad_id = vocab.word2id(PAD_TOKEN) # id of the PAD token used to pad sequences
    self.init_encoder_seq(example_list) # initialize the input to the encoder
    self.init_decoder_seq(example_list) # initialize the input and targets for the decoder
    self.store_orig_strings(example_list) # store the original strings


  def init_encoder_seq(self, example_list):
    # Determine the maximum length of the encoder input sequence in this batch
    max_enc_seq_len = max([ex.enc_len for ex in example_list])

    # Pad the encoder input sequences up to the length of the longest sequence
    for ex in example_list:
      ex.pad_encoder_input(max_enc_seq_len, self.pad_id)

    # Initialize the numpy arrays
    # Note: our enc_batch can have different length (second dimension) for each batch because we use dynamic_rnn for the encoder.
    self.enc_batch = np.zeros((self.batch_size, max_enc_seq_len), dtype=np.int32)
    self.enc_lens = np.zeros((self.batch_size), dtype=np.int32)
    self.enc_padding_mask = np.zeros((self.batch_size, max_enc_seq_len), dtype=np.float32)

    # Fill in the numpy arrays
    for i, ex in enumerate(example_list):
      self.enc_batch[i, :] = ex.enc_input[:]
      self.enc_lens[i] = ex.enc_len
      for j in range(ex.enc_len):
        self.enc_padding_mask[i][j] = 1

    # For pointer-generator mode, need to store some extra info
    if pointer_gen:
      # Determine the max number of in-article OOVs in this batch
      self.max_art_oovs = max([len(ex.article_oovs) for ex in example_list])
      # Store the in-article OOVs themselves
      self.art_oovs = [ex.article_oovs for ex in example_list]
      # Store the version of the enc_batch that uses the article OOV ids
      self.enc_batch_extend_vocab = np.zeros((self.batch_size, max_enc_seq_len), dtype=np.int32)
      for i, ex in enumerate(example_list):
        self.enc_batch_extend_vocab[i, :] = ex.enc_input_extend_vocab[:]

  def init_decoder_seq(self, example_list):
    # Pad the inputs and targets
    for ex in example_list:
      ex.pad_decoder_inp_targ(max_dec_steps, self.pad_id)

    # Initialize the numpy arrays.
    self.dec_batch = np.zeros((self.batch_size, max_dec_steps), dtype=np.int32)
    self.target_batch = np.zeros((self.batch_size, max_dec_steps), dtype=np.int32)
    self.dec_padding_mask = np.zeros((self.batch_size, max_dec_steps), dtype=np.float32)
    self.dec_lens = np.zeros((self.batch_size), dtype=np.int32)

    # Fill in the numpy arrays
    for i, ex in enumerate(example_list):
      self.dec_batch[i, :] = ex.dec_input[:]
      self.target_batch[i, :] = ex.target[:]
      self.dec_lens[i] = ex.dec_len
      for j in range(ex.dec_len):
        self.dec_padding_mask[i][j] = 1

  def store_orig_strings(self, example_list):
    self.original_articles = [ex.original_article for ex in example_list] # list of lists
    self.original_abstracts = [ex.original_abstract for ex in example_list] # list of lists
    self.original_abstracts_sents = [ex.original_abstract_sents for ex in example_list] # list of list of lists

In [0]:
voccy.

In [0]:
train_data_path = "./tokenized/train/" +"*.txt"

batches = Batcher(train_data_path, voccy, mode='train',
                               batch_size=8, single_pass=False)

INFO:tensorflow:Bucket queue size: 0, Input queue size: 0
Yielding

Exception in thread Thread-30:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-123-400d7f2080ca>", line 70, in fill_example_queue
    abstract_sentences = [sent.strip() for sent in abstract2sents(abstract)]
  File "<ipython-input-1-d99556ea03ab>", line 161, in abstract2sents
    start_p = abstract.index(SENTENCE_START, cur)
TypeError: a bytes-like object is required, not 'str'


In [0]:
#@title Batcher Class
class Batcher(object):
  BATCH_QUEUE_MAX = 100 # max number of batches the batch_queue can hold

  def __init__(self, data_path, vocab, mode, batch_size, single_pass):
    #self._data_path = data_path
    self._vocab = vocab
    self._single_pass = single_pass
    self.mode = mode
    self.batch_size = batch_size
    # Initialize a queue of Batches waiting to be used, and a queue of Examples waiting to be batched
    self._batch_queue = Queue.Queue(self.BATCH_QUEUE_MAX)
    self._example_queue = Queue.Queue(self.BATCH_QUEUE_MAX * self.batch_size)

    # Different settings depending on whether we're in single_pass mode or not
    if single_pass:
      self._num_example_q_threads = 1 # just one thread, so we read through the dataset just once
      self._num_batch_q_threads = 1  # just one thread to batch examples
      self._bucketing_cache_size = 1 # only load one batch's worth of examples before bucketing; this essentially means no bucketing
      self._finished_reading = False # this will tell us when we're finished reading the dataset
    else:
      self._num_example_q_threads = 1 #16 # num threads to fill example queue
      self._num_batch_q_threads = 1 #4  # num threads to fill batch queue
      self._bucketing_cache_size = 1 #100 # how many batches-worth of examples to load into cache before bucketing

    # Start the threads that load the queues
    self._example_q_threads = []
    for _ in range(self._num_example_q_threads):
      self._example_q_threads.append(Thread(target=self.fill_example_queue))
      self._example_q_threads[-1].daemon = True
      self._example_q_threads[-1].start()
    self._batch_q_threads = []
    for _ in range(self._num_batch_q_threads):
      self._batch_q_threads.append(Thread(target=self.fill_batch_queue))
      self._batch_q_threads[-1].daemon = True
      self._batch_q_threads[-1].start()

    # Start a thread that watches the other threads and restarts them if they're dead
    if not single_pass: # We don't want a watcher in single_pass mode because the threads shouldn't run forever
      self._watch_thread = Thread(target=self.watch_threads)
      self._watch_thread.daemon = True
      self._watch_thread.start()

  def next_batch(self):
    # If the batch queue is empty, print a warning
    if self._batch_queue.qsize() == 0:
      tf.logging.warning('Bucket input queue is empty when calling next_batch. Bucket queue size: %i, Input queue size: %i', self._batch_queue.qsize(), self._example_queue.qsize())
      if self._single_pass and self._finished_reading:
        tf.logging.info("Finished reading dataset in single_pass mode.")
        return None

    batch = self._batch_queue.get() # get the next Batch
    return batch

  def fill_example_queue(self):
    input_gen = self.text_generator(example_generator(self._data_path, self._single_pass))

    while True:
      try:
        (article, abstract) = next(input_gen) # read the next example from file. article and abstract are both strings.
      except StopIteration: # if there are no more examples:
        tf.logging.info("The example generator for this example queue filling thread has exhausted data.")
        if self._single_pass:
          tf.logging.info("single_pass mode is on, so we've finished reading dataset. This thread is stopping.")
          self._finished_reading = True
          break
        else:
          raise Exception("single_pass mode is off but the example generator is out of data; error.")

      #abstract_sentences = [sent.strip() for sent in data.abstract2sents(abstract)] # Use the <s> and </s> tags in abstract to get a list of sentences.
      abstract_sentences = [sent.strip() for sent in abstract2sents(abstract)]
      example = Example(article, abstract_sentences, self._vocab) # Process into an Example.
      self._example_queue.put(example) # place the Example in the example queue.

  def fill_batch_queue(self):
    while True:
      if self.mode == 'decode':
        # beam search decode mode single example repeated in the batch
        ex = self._example_queue.get()
        b = [ex for _ in range(self.batch_size)]
        self._batch_queue.put(Batch(b, self._vocab, self.batch_size))
      else:
        # Get bucketing_cache_size-many batches of Examples into a list, then sort
        inputs = []
        for _ in range(self.batch_size * self._bucketing_cache_size):
          inputs.append(self._example_queue.get())
        inputs = sorted(inputs, key=lambda inp: inp.enc_len, reverse=True) # sort by length of encoder sequence

        # Group the sorted Examples into batches, optionally shuffle the batches, and place in the batch queue.
        batches = []
        for i in range(0, len(inputs), self.batch_size):
          batches.append(inputs[i:i + self.batch_size])
        if not self._single_pass:
          shuffle(batches)
        for b in batches:  # each b is a list of Example objects
          self._batch_queue.put(Batch(b, self._vocab, self.batch_size))

  def watch_threads(self):
    while True:
      tf.logging.info(
        'Bucket queue size: %i, Input queue size: %i',
        self._batch_queue.qsize(), self._example_queue.qsize())

      time.sleep(60)
      for idx,t in enumerate(self._example_q_threads):
        if not t.is_alive(): # if the thread is dead
          tf.logging.error('Found example queue thread dead. Restarting.')
          new_t = Thread(target=self.fill_example_queue)
          self._example_q_threads[idx] = new_t
          new_t.daemon = True
          new_t.start()
      for idx,t in enumerate(self._batch_q_threads):
        if not t.is_alive(): # if the thread is dead
          tf.logging.error('Found batch queue thread dead. Restarting.')
          new_t = Thread(target=self.fill_batch_queue)
          self._batch_q_threads[idx] = new_t
          new_t.daemon = True
          new_t.start()


  def text_generator(self, example_generator):
    while True:
      e = next(example_generator) # e is a tf.Example
      try:
        article_text = e.features.feature['article'].bytes_list.value[0] # the article text was saved under the key 'article' in the data files
        abstract_text = e.features.feature['abstract'].bytes_list.value[0] # the abstract text was saved under the key 'abstract' in the data files
      except ValueError:
        tf.logging.error('Failed to get article or abstract from example')
        continue
      if len(article_text)==0: # See https://github.com/abisee/pointer-generator/issues/1
        #tf.logging.warning('Found an example with empty article text. Skipping it.')
        continue
      else:
        yield (article_text, abstract_text)

In [0]:
################BATCHER~~~~END############

In [0]:
################MODEL###############

In [0]:
#@title
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from numpy import random

use_cuda = torch.cuda.is_available()

random.seed(123)
torch.manual_seed(123)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(123)

def init_lstm_wt(lstm):
    for names in lstm._all_weights:
        for name in names:
            if name.startswith('weight_'):
                wt = getattr(lstm, name)
                wt.data.uniform_(-rand_unif_init_mag, rand_unif_init_mag)
            elif name.startswith('bias_'):
                # set forget bias to 1
                bias = getattr(lstm, name)
                n = bias.size(0)
                start, end = n // 4, n // 2
                bias.data.fill_(0.)
                bias.data[start:end].fill_(1.)

def init_linear_wt(linear):
    linear.weight.data.normal_(std=trunc_norm_init_std)
    if linear.bias is not None:
        linear.bias.data.normal_(std=trunc_norm_init_std)

def init_wt_normal(wt):
    wt.data.normal_(std=trunc_norm_init_std)

def init_wt_unif(wt):
    wt.data.uniform_(-rand_unif_init_mag, rand_unif_init_mag)

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        init_wt_normal(self.embedding.weight)

        self.lstm = nn.LSTM(emb_dim, hidden_dim, num_layers=1, batch_first=True, bidirectional=True)
        init_lstm_wt(self.lstm)

        self.W_h = nn.Linear(hidden_dim * 2, hidden_dim * 2, bias=False)

    #seq_lens should be in descending order
    def forward(self, input, seq_lens):
        embedded = self.embedding(input)

        packed = pack_padded_sequence(embedded, seq_lens, batch_first=True)
        output, hidden = self.lstm(packed)

        encoder_outputs, _ = pad_packed_sequence(output, batch_first=True)  # h dim = B x t_k x n
        encoder_outputs = encoder_outputs.contiguous()

        encoder_feature = encoder_outputs.view(-1, 2*hidden_dim)  # B * t_k x 2*hidden_dim
        encoder_feature = self.W_h(encoder_feature)

        return encoder_outputs, encoder_feature, hidden

class ReduceState(nn.Module):
    def __init__(self):
        super(ReduceState, self).__init__()

        self.reduce_h = nn.Linear(hidden_dim * 2, hidden_dim)
        init_linear_wt(self.reduce_h)
        self.reduce_c = nn.Linear(hidden_dim * 2, hidden_dim)
        init_linear_wt(self.reduce_c)

    def forward(self, hidden):
        h, c = hidden # h, c dim = 2 x b x hidden_dim
        h_in = h.transpose(0, 1).contiguous().view(-1, hidden_dim * 2)
        hidden_reduced_h = F.relu(self.reduce_h(h_in))
        c_in = c.transpose(0, 1).contiguous().view(-1, hidden_dim * 2)
        hidden_reduced_c = F.relu(self.reduce_c(c_in))

        return (hidden_reduced_h.unsqueeze(0), hidden_reduced_c.unsqueeze(0)) # h, c dim = 1 x b x hidden_dim

class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        # attention
        if is_coverage:
            self.W_c = nn.Linear(1, hidden_dim * 2, bias=False)
        self.decode_proj = nn.Linear(hidden_dim * 2, hidden_dim * 2)
        self.v = nn.Linear(hidden_dim * 2, 1, bias=False)

    def forward(self, s_t_hat, encoder_outputs, encoder_feature, enc_padding_mask, coverage):
        b, t_k, n = list(encoder_outputs.size())

        dec_fea = self.decode_proj(s_t_hat) # B x 2*hidden_dim
        dec_fea_expanded = dec_fea.unsqueeze(1).expand(b, t_k, n).contiguous() # B x t_k x 2*hidden_dim
        dec_fea_expanded = dec_fea_expanded.view(-1, n)  # B * t_k x 2*hidden_dim

        att_features = encoder_feature + dec_fea_expanded # B * t_k x 2*hidden_dim
        if is_coverage:
            coverage_input = coverage.view(-1, 1)  # B * t_k x 1
            coverage_feature = self.W_c(coverage_input)  # B * t_k x 2*hidden_dim
            att_features = att_features + coverage_feature

        e = F.tanh(att_features) # B * t_k x 2*hidden_dim
        scores = self.v(e)  # B * t_k x 1
        scores = scores.view(-1, t_k)  # B x t_k

        attn_dist_ = F.softmax(scores, dim=1)*enc_padding_mask # B x t_k
        normalization_factor = attn_dist_.sum(1, keepdim=True)
        attn_dist = attn_dist_ / normalization_factor

        attn_dist = attn_dist.unsqueeze(1)  # B x 1 x t_k
        c_t = torch.bmm(attn_dist, encoder_outputs)  # B x 1 x n
        c_t = c_t.view(-1, hidden_dim * 2)  # B x 2*hidden_dim

        attn_dist = attn_dist.view(-1, t_k)  # B x t_k

        if is_coverage:
            coverage = coverage.view(-1, t_k)
            coverage = coverage + attn_dist

        return c_t, attn_dist, coverage

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.attention_network = Attention()
        # decoder
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        init_wt_normal(self.embedding.weight)

        self.x_context = nn.Linear(hidden_dim * 2 + emb_dim, emb_dim)

        self.lstm = nn.LSTM(emb_dim, hidden_dim, num_layers=1, batch_first=True, bidirectional=False)
        init_lstm_wt(self.lstm)

        if pointer_gen:
            self.p_gen_linear = nn.Linear(hidden_dim * 4 + emb_dim, 1)

        #p_vocab
        self.out1 = nn.Linear(hidden_dim * 3, hidden_dim)
        self.out2 = nn.Linear(hidden_dim, vocab_size)
        init_linear_wt(self.out2)

    def forward(self, y_t_1, s_t_1, encoder_outputs, encoder_feature, enc_padding_mask,
                c_t_1, extra_zeros, enc_batch_extend_vocab, coverage, step):

        if not self.training and step == 0:
            h_decoder, c_decoder = s_t_1
            s_t_hat = torch.cat((h_decoder.view(-1, hidden_dim),
                                 c_decoder.view(-1, hidden_dim)), 1)  # B x 2*hidden_dim
            c_t, _, coverage_next = self.attention_network(s_t_hat, encoder_outputs, encoder_feature,
                                                              enc_padding_mask, coverage)
            coverage = coverage_next

        y_t_1_embd = self.embedding(y_t_1)
        x = self.x_context(torch.cat((c_t_1, y_t_1_embd), 1))
        lstm_out, s_t = self.lstm(x.unsqueeze(1), s_t_1)

        h_decoder, c_decoder = s_t
        s_t_hat = torch.cat((h_decoder.view(-1, hidden_dim),
                             c_decoder.view(-1, hidden_dim)), 1)  # B x 2*hidden_dim
        c_t, attn_dist, coverage_next = self.attention_network(s_t_hat, encoder_outputs, encoder_feature,
                                                          enc_padding_mask, coverage)

        if self.training or step > 0:
            coverage = coverage_next

        p_gen = None
        if pointer_gen:
            p_gen_input = torch.cat((c_t, s_t_hat, x), 1)  # B x (2*2*hidden_dim + emb_dim)
            p_gen = self.p_gen_linear(p_gen_input)
            p_gen = F.sigmoid(p_gen)

        output = torch.cat((lstm_out.view(-1, hidden_dim), c_t), 1) # B x hidden_dim * 3
        output = self.out1(output) # B x hidden_dim

        #output = F.relu(output)

        output = self.out2(output) # B x vocab_size
        vocab_dist = F.softmax(output, dim=1)

        if pointer_gen:
            vocab_dist_ = p_gen * vocab_dist
            attn_dist_ = (1 - p_gen) * attn_dist

            if extra_zeros is not None:
                vocab_dist_ = torch.cat([vocab_dist_, extra_zeros], 1)

            final_dist = vocab_dist_.scatter_add(1, enc_batch_extend_vocab, attn_dist_)
        else:
            final_dist = vocab_dist

        return final_dist, s_t, c_t, attn_dist, p_gen, coverage

class Model(object):
    def __init__(self, model_file_path=None, is_eval=False):
        encoder = Encoder()
        decoder = Decoder()
        reduce_state = ReduceState()

        # shared the embedding between encoder and decoder
        decoder.embedding.weight = encoder.embedding.weight
        if is_eval:
            encoder = encoder.eval()
            decoder = decoder.eval()
            reduce_state = reduce_state.eval()

        if use_cuda:
            encoder = encoder.cuda()
            decoder = decoder.cuda()
            reduce_state = reduce_state.cuda()

        self.encoder = encoder
        self.decoder = decoder
        self.reduce_state = reduce_state

        if model_file_path is not None:
            state = torch.load(model_file_path, map_location= lambda storage, location: storage)
            self.encoder.load_state_dict(state['encoder_state_dict'])
            self.decoder.load_state_dict(state['decoder_state_dict'], strict=False)
            self.reduce_state.load_state_dict(state['reduce_state_dict'])

In [0]:
################MODEL~~~END###############

In [0]:
##############TRAIN########################

Need to create a batcher that we can get_next()

In [0]:
class Train(object):
    def __init__(self, vocab_fn):
        self.vocab_fn = vocab_fn
        self.vocab = Vocab(vocab_path, vocab_size, vocab_fn)
        self.batcher = Batcher(train_data_path, self.vocab, mode='train',
                               batch_size=batch_size, single_pass=False)
        time.sleep(15)
        #if dir doesnt exist create it
        train_dir = os.path.join(log_root, 'train_%d' % (int(time.time())))
        if not os.path.exists(train_dir):
            os.mkdir(train_dir)
        #if dir doesnt exist create it
        self.model_dir = os.path.join(train_dir, 'model')
        if not os.path.exists(self.model_dir):
            os.mkdir(self.model_dir)
      
        self.summary_writer = tf.summary.FileWriter(train_dir)

    def save_model(self, running_avg_loss, iter):
        state = {
            'iter': iter,
            'encoder_state_dict': self.model.encoder.state_dict(),
            'decoder_state_dict': self.model.decoder.state_dict(),
            'reduce_state_dict': self.model.reduce_state.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'current_loss': running_avg_loss
        }
        model_save_path = os.path.join(self.model_dir, 'model_%d_%d' % (iter, int(time.time())))
        torch.save(state, model_save_path)

    def setup_train(self, model_file_path=None):
        self.model = Model(model_file_path)

        params = list(self.model.encoder.parameters()) + list(self.model.decoder.parameters()) + \
                 list(self.model.reduce_state.parameters())
        initial_lr = lr_coverage if is_coverage else lr
        self.optimizer = Adagrad(params, lr=initial_lr, initial_accumulator_value=adagrad_init_acc)

        start_iter, start_loss = 0, 0

        if model_file_path is not None:
            state = torch.load(model_file_path, map_location= lambda storage, location: storage)
            start_iter = state['iter']
            start_loss = state['current_loss']

            if not is_coverage:
                self.optimizer.load_state_dict(state['optimizer'])
                if use_cuda:
                    for state in self.optimizer.state.values():
                        for k, v in state.items():
                            if torch.is_tensor(v):
                                state[k] = v.cuda()

        return start_iter, start_loss

    def train_one_batch(self, batch):
        enc_batch, enc_padding_mask, enc_lens, enc_batch_extend_vocab, extra_zeros, c_t_1, coverage = \
            get_input_from_batch(batch, use_cuda)
        dec_batch, dec_padding_mask, max_dec_len, dec_lens_var, target_batch = \
            get_output_from_batch(batch, use_cuda)

        self.optimizer.zero_grad()

        encoder_outputs, encoder_feature, encoder_hidden = self.model.encoder(enc_batch, enc_lens)
        s_t_1 = self.model.reduce_state(encoder_hidden)

        step_losses = []
        for di in range(min(max_dec_len, max_dec_steps)):
            y_t_1 = dec_batch[:, di]  # Teacher forcing
            final_dist, s_t_1,  c_t_1, attn_dist, p_gen, next_coverage = self.model.decoder(y_t_1, s_t_1,
                                                        encoder_outputs, encoder_feature, enc_padding_mask, c_t_1,
                                                        extra_zeros, enc_batch_extend_vocab,
                                                                           coverage, di)
            target = target_batch[:, di]
            gold_probs = torch.gather(final_dist, 1, target.unsqueeze(1)).squeeze()
            step_loss = -torch.log(gold_probs + eps)
            if is_coverage:
                step_coverage_loss = torch.sum(torch.min(attn_dist, coverage), 1)
                step_loss = step_loss + cov_loss_wt * step_coverage_loss
                coverage = next_coverage
                
            step_mask = dec_padding_mask[:, di]
            step_loss = step_loss * step_mask
            step_losses.append(step_loss)

        sum_losses = torch.sum(torch.stack(step_losses, 1), 1)
        batch_avg_loss = sum_losses/dec_lens_var
        loss = torch.mean(batch_avg_loss)

        loss.backward()

        self.norm = clip_grad_norm_(self.model.encoder.parameters(), max_grad_norm)
        clip_grad_norm_(self.model.decoder.parameters(), max_grad_norm)
        clip_grad_norm_(self.model.reduce_state.parameters(), max_grad_norm)

        self.optimizer.step()

        return loss.item()

    def trainIters(self, n_iters, model_file_path=None):
        iter, running_avg_loss = self.setup_train(model_file_path)
        start = time.time()
        while iter < n_iters:
            batch = self.batcher.next_batch()
            loss = self.train_one_batch(batch)

            running_avg_loss = calc_running_avg_loss(loss, running_avg_loss, self.summary_writer, iter)
            iter += 1

            if iter % 100 == 0:
                self.summary_writer.flush()
            print_interval = 1000
            if iter % print_interval == 0:
                print('steps %d, seconds for %d batch: %.2f , loss: %f' % (iter, print_interval,
                                                                           time.time() - start, loss))
                start = time.time()
            if iter % 5000 == 0:
                self.save_model(running_avg_loss, iter)


#Arguments("-m", dest="model_file_path", required=False, default=None, Model file for retraining (default: None).")
model_file_path = './'
if model_file_path is not '':
  train_processor = Train(voc_txt)
  train_processor.trainIters(max_iterations, model_file_path)

NameError: ignored

In [0]:
##############TRAIN~~END########################

In [0]:
##############DECODE########################

In [0]:
class Beam(object):
  def __init__(self, tokens, log_probs, state, context, coverage):
    self.tokens = tokens
    self.log_probs = log_probs
    self.state = state
    self.context = context
    self.coverage = coverage

  def extend(self, token, log_prob, state, context, coverage):
    return Beam(tokens = self.tokens + [token],
                      log_probs = self.log_probs + [log_prob],
                      state = state,
                      context = context,
                      coverage = coverage)

  @property
  def latest_token(self):
    return self.tokens[-1]

  @property
  def avg_log_prob(self):
    return sum(self.log_probs) / len(self.tokens)


class BeamSearch(object):
    def __init__(self, model_file_path):
        model_name = os.path.basename(model_file_path)
        self._decode_dir = os.path.join(log_root, 'decode_%s' % (model_name))
        self._rouge_ref_dir = os.path.join(self._decode_dir, 'rouge_ref')
        self._rouge_dec_dir = os.path.join(self._decode_dir, 'rouge_dec_dir')
        for p in [self._decode_dir, self._rouge_ref_dir, self._rouge_dec_dir]:
            if not os.path.exists(p):
                os.mkdir(p)

        self.vocab = Vocab(vocab_path, vocab_size)
        self.batcher = Batcher(decode_data_path, self.vocab, mode='decode',
                               batch_size=beam_size, single_pass=True)
        time.sleep(15)

        self.model = Model(model_file_path, is_eval=True)

    def sort_beams(self, beams):
        return sorted(beams, key=lambda h: h.avg_log_prob, reverse=True)


    def decode(self):
        start = time.time()
        counter = 0
        batch = self.batcher.next_batch()
        while batch is not None:
            # Run beam search to get best Hypothesis
            best_summary = self.beam_search(batch)

            # Extract the output ids from the hypothesis and convert back to words
            output_ids = [int(t) for t in best_summary.tokens[1:]]
            decoded_words = data.outputids2words(output_ids, self.vocab,
                                                 (batch.art_oovs[0] if pointer_gen else None))

            # Remove the [STOP] token from decoded_words, if necessary
            try:
                fst_stop_idx = decoded_words.index(data.STOP_DECODING)
                decoded_words = decoded_words[:fst_stop_idx]
            except ValueError:
                decoded_words = decoded_words

            original_abstract_sents = batch.original_abstracts_sents[0]

            write_for_rouge(original_abstract_sents, decoded_words, counter,
                            self._rouge_ref_dir, self._rouge_dec_dir)
            counter += 1
            if counter % 1000 == 0:
                print('%d example in %d sec'%(counter, time.time() - start))
                start = time.time()

            batch = self.batcher.next_batch()

        print("Decoder has finished reading dataset for single_pass.")
        print("Now starting ROUGE eval...")
        results_dict = rouge_eval(self._rouge_ref_dir, self._rouge_dec_dir)
        rouge_log(results_dict, self._decode_dir)


    def beam_search(self, batch):
        #batch should have only one example
        enc_batch, enc_padding_mask, enc_lens, enc_batch_extend_vocab, extra_zeros, c_t_0, coverage_t_0 = \
            get_input_from_batch(batch, use_cuda)

        encoder_outputs, encoder_feature, encoder_hidden = self.model.encoder(enc_batch, enc_lens)
        s_t_0 = self.model.reduce_state(encoder_hidden)

        dec_h, dec_c = s_t_0 # 1 x 2*hidden_size
        dec_h = dec_h.squeeze()
        dec_c = dec_c.squeeze()

        #decoder batch preparation, it has beam_size example initially everything is repeated
        beams = [Beam(tokens=[self.vocab.word2id(data.START_DECODING)],
                      log_probs=[0.0],
                      state=(dec_h[0], dec_c[0]),
                      context = c_t_0[0],
                      coverage=(coverage_t_0[0] if is_coverage else None))
                 for _ in xrange(beam_size)]
        results = []
        steps = 0
        while steps < max_dec_steps and len(results) < beam_size:
            latest_tokens = [h.latest_token for h in beams]
            latest_tokens = [t if t < self.vocab.size() else self.vocab.word2id(data.UNKNOWN_TOKEN) \
                             for t in latest_tokens]
            y_t_1 = Variable(torch.LongTensor(latest_tokens))
            if use_cuda:
                y_t_1 = y_t_1.cuda()
            all_state_h =[]
            all_state_c = []

            all_context = []

            for h in beams:
                state_h, state_c = h.state
                all_state_h.append(state_h)
                all_state_c.append(state_c)

                all_context.append(h.context)

            s_t_1 = (torch.stack(all_state_h, 0).unsqueeze(0), torch.stack(all_state_c, 0).unsqueeze(0))
            c_t_1 = torch.stack(all_context, 0)

            coverage_t_1 = None
            if is_coverage:
                all_coverage = []
                for h in beams:
                    all_coverage.append(h.coverage)
                coverage_t_1 = torch.stack(all_coverage, 0)

            final_dist, s_t, c_t, attn_dist, p_gen, coverage_t = self.model.decoder(y_t_1, s_t_1,
                                                        encoder_outputs, encoder_feature, enc_padding_mask, c_t_1,
                                                        extra_zeros, enc_batch_extend_vocab, coverage_t_1, steps)
            log_probs = torch.log(final_dist)
            topk_log_probs, topk_ids = torch.topk(log_probs, beam_size * 2)

            dec_h, dec_c = s_t
            dec_h = dec_h.squeeze()
            dec_c = dec_c.squeeze()

            all_beams = []
            num_orig_beams = 1 if steps == 0 else len(beams)
            for i in xrange(num_orig_beams):
                h = beams[i]
                state_i = (dec_h[i], dec_c[i])
                context_i = c_t[i]
                coverage_i = (coverage_t[i] if is_coverage else None)

                for j in xrange(beam_size * 2):  # for each of the top 2*beam_size hyps:
                    new_beam = h.extend(token=topk_ids[i, j].item(),
                                   log_prob=topk_log_probs[i, j].item(),
                                   state=state_i,
                                   context=context_i,
                                   coverage=coverage_i)
                    all_beams.append(new_beam)

            beams = []
            for h in self.sort_beams(all_beams):
                if h.latest_token == self.vocab.word2id(data.STOP_DECODING):
                    if steps >= min_dec_steps:
                        results.append(h)
                else:
                    beams.append(h)
                if len(beams) == beam_size or len(results) == beam_size:
                    break

            steps += 1

        if len(results) == 0:
            results = beams

        beams_sorted = self.sort_beams(results)

        return beams_sorted[0]

model_filename = ''
if model_filename is not '':
  beam_Search_processor = BeamSearch(model_filename)
  beam_Search_processor.decode()
else:
  pass

In [0]:
####################DECODE~~~END####################

In [0]:
###############TRAIN UTIL###########

In [0]:
def get_input_from_batch(batch, use_cuda):
  batch_size = len(batch.enc_lens)

  enc_batch = Variable(torch.from_numpy(batch.enc_batch).long())
  enc_padding_mask = Variable(torch.from_numpy(batch.enc_padding_mask)).float()
  enc_lens = batch.enc_lens
  extra_zeros = None
  enc_batch_extend_vocab = None

  if pointer_gen:
    enc_batch_extend_vocab = Variable(torch.from_numpy(batch.enc_batch_extend_vocab).long())
    # max_art_oovs is the max over all the article oov list in the batch
    if batch.max_art_oovs > 0:
      extra_zeros = Variable(torch.zeros((batch_size, batch.max_art_oovs)))

  c_t_1 = Variable(torch.zeros((batch_size, 2 * hidden_dim)))

  coverage = None
  if is_coverage:
    coverage = Variable(torch.zeros(enc_batch.size()))

  if use_cuda:
    enc_batch = enc_batch.cuda()
    enc_padding_mask = enc_padding_mask.cuda()

    if enc_batch_extend_vocab is not None:
      enc_batch_extend_vocab = enc_batch_extend_vocab.cuda()
    if extra_zeros is not None:
      extra_zeros = extra_zeros.cuda()
    c_t_1 = c_t_1.cuda()

    if coverage is not None:
      coverage = coverage.cuda()

  return enc_batch, enc_padding_mask, enc_lens, enc_batch_extend_vocab, extra_zeros, c_t_1, coverage

def get_output_from_batch(batch, use_cuda):
  dec_batch = Variable(torch.from_numpy(batch.dec_batch).long())
  dec_padding_mask = Variable(torch.from_numpy(batch.dec_padding_mask)).float()
  dec_lens = batch.dec_lens
  max_dec_len = np.max(dec_lens)
  dec_lens_var = Variable(torch.from_numpy(dec_lens)).float()

  target_batch = Variable(torch.from_numpy(batch.target_batch)).long()

  if use_cuda:
    dec_batch = dec_batch.cuda()
    dec_padding_mask = dec_padding_mask.cuda()
    dec_lens_var = dec_lens_var.cuda()
    target_batch = target_batch.cuda()


  return dec_batch, dec_padding_mask, max_dec_len, dec_lens_var, target_batch

In [0]:
###############TRAIN UTIL~~~~END###########

In [0]:
################To do Model Transformer###########

In [0]:
random.seed(123)
torch.manual_seed(123)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(123)

d_model = 512
N = 6
H = 8
d_k = d_v = 64 # = d_model / H
sqrt_d_k = np.sqrt(d_k)

d_ff = 2048

def get_pos_embedding(max_len):
    pos_emb = np.zeros(max_len, d_model)
    pos = np.arange(0, max_len)

    denom = np.exp(np.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))

    pos_emb[:, 0::2] = np.sin(pos*denom)
    pos_emb[:, 1::2] = np.cos(pos*denom)

    return pos_emb

def init_wt_normal(wt):
    wt.data.normal_(std=trunc_norm_init_std)

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        init_wt_normal(self.embedding.weight)

        layers = [EncoderLayer() for _ in range(N)]
        features = nn.Sequential(*layers)

    def forward(self, input, seq_lens):
        embedded = self.embedding(input) * np.sqrt(d_model)


class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.multi_head_att = MultiHeadAttention()
        self.ln_mh = nn.LayerNorm([d_model])

        self.affine1 = nn.Linear(d_model, d_ff, bias=True)
        self.affine2 = nn.Linear(d_ff, d_model, bias=True)
        self.ln_aff = nn.LayerNorm([d_model])

    def forward(self, x):
        x_att = self.multi_head_att(x)
        x_1 = self.ln_mh(x + x_att)

        x_aff = F.relu(self.affine1(x_1))
        x_aff = self.affine2(x_aff)
        x_2 = self.ln_aff(x_1 + x_aff)

        return x_2

class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_model, bias=False)
        self.W_K = nn.Linear(d_model, d_model, bias=False)
        self.W_V = nn.Linear(d_model, d_model, bias=False)

        self.W_O = nn.Linear(d_model, d_model, bias=False)

    def forward(self, q, k, v):
        q_proj = self.W_Q(q) #B x d_model
        k_proj = self.W_K(k)
        v_proj = self.W_V(v)

        q_proj_split = torch.split(q_proj, split_size_or_sections=d_k, dim=1)
        k_proj_split = torch.split(k_proj, split_size_or_sections=d_k, dim=1)
        v_proj_split = torch.split(v_proj, split_size_or_sections=d_v, dim=1)

        heads = []

        for i, q_i in enumerate(q_proj_split):
            k_i = k_proj_split[i] #B x d_k
            v_i = v_proj_split[i]
            qk_i = torch.bmm(q_i, k_i)/sqrt_d_k ##B x d_k
            att_i = F.softmax(qk_i)
            att_i = torch.bmm(att_i, v_i)

            heads.append(att_i)

        heads_cat = torch.cat(heads, dim=1)
        multi_head_att = self.W_O(heads_cat)

        return multi_head_att

In [0]:
################To do Model Transformer~~~END###########